In [ ]:
# import python packages and set plotting configurations 
import pynbody as pb
import numpy as np
import pynbody.filt as filt
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
import pickle as p
import glob
import os

%matplotlib inline
#plt.rcParams['figure.figsize'] = (10, 10)

plt.rcParams['figure.figsize'] = (10, 10)

sns.set_style('ticks')
#sns.set_style('darkgrid')
sns.set_context("talk",font_scale=2,rc={"lines.linewidth": 4,"axes.linewidth": 5})

plt.rc('axes', linewidth=3)
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['xtick.top'] = True
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['ytick.right'] = True
plt.rcParams['xtick.major.size'] = 10
plt.rcParams['xtick.major.width'] = 3
plt.rcParams['xtick.minor.size'] = 5
plt.rcParams['xtick.minor.width'] = 1.5
plt.rcParams['ytick.major.size'] = 10
plt.rcParams['ytick.major.width'] = 3
plt.rcParams['ytick.minor.size'] = 5
plt.rcParams['ytick.minor.width'] = 1.5
plt.rcParams['axes.edgecolor'] = 'k'#'gray'
#plt.rcParams['axes.grid'] = True
#plt.rcParams['grid.color'] = 'lightgray'
#plt.rcParams['grid.linestyle'] = 'dashed' #dashes=(5, 1)
#plt.rcParams['lines.dashed_pattern'] = 10, 3
#plt.rcParams['grid.linewidth'] = 1.5
#plt.rcParams['axes.facecolor'] = 'whitesmoke'
#plt.rcParams['axes.axisbelow'] = True
plt.rcParams['legend.fancybox'] = True
plt.rcParams['legend.frameon'] = True
plt.rcParams['legend.shadow'] = False
plt.rcParams['legend.edgecolor'] = 'lightgray'
plt.rcParams['patch.linewidth'] = 3

gal = filt.LowPass('r', '25 kpc')
disk = filt.Disc('25 kpc', '4 kpc')

In [ ]:
from astroML.correlation import two_point

In [ ]:
sims = ['master', 'padoan', 'semenov', 'evans', 'federrath']

In [ ]:
bins = np.linspace(0, 10, 41) #could use smaller bins, say down to 200pc and using 41 bins...

In [ ]:
bins

In [ ]:
def power_law(x, x0, gamma, a):
    '''Normal power law to fit.'''
    return a * (x/x0)**(-gamma)

def log_power_law(x, gamma, x0, a):
    '''Power law in log log space to fit'''
    return -gamma * a * (x - x0)

def schechter(x, gamma, x0, a):
    '''Schechter like fit (power-law with exponential cut-off 
    to account for the finite extension of the stellar disc)'''
    return a * ((x/x0)**-gamma)*np.exp(-(x/x0))

def log_schechter(x, gamma, x0, a):
    '''Schechter like fit'''
    return a - gamma * (x - x0) - (x/x0)/np.log(10)

In [ ]:
# r_half is the half mass radius of the stellar disc
10**np.linspace(0, np.ceil(np.log10(3*data['r_half'])), np.floor(3*data['r_half']/0.2)+1)

In [ ]:
sims_good = ['g1.37e11','g1.57e11','g1.59e11','g3.49e11','g3.55e11','g3.71e11','g7.08e11','g7.55e11','g7.66e11','g8.26e11'] #,'g2.19e11','g3.21e11'

In [ ]:
# only redshift 0 snaps
from scipy.optimize import curve_fit
#[[0.3,50.0,0.65],[0.0,25.0,1.0],[0.07,100.0,0.75],[0.3,50.0,0.65]],

p_starts = [[[0.1,380.0,0.6],[0.1,200.0,0.5],[0.1,70.0,0.5],[0.1,10000.0,0.3]],
            [[0.09,34.,.95],[0.16,20.0,0.9],[0.0,20.0,0.9]],
            [[0.3,50.0,0.65],[0.3,50.0,0.65],[0.3,50.0,0.65],[0.2,15.0,0.8],[0.3,40.0,.35]],
            [[0.1,55.0,0.8],[0.1,55.0,0.8],[0.1,55.0,0.8]],
            [[0.15,40.0,0.84],[0.06,102.0,1.05],[0.39,25.2,.84]],
            [[0.1,30.0,1.0],[0.1,30.0,1.0],[0.3,40.0,0.45]],
            [[0.0,10.0,1.7],[0.04,60.22,0.92],[0.0,10.0,1.5],[0.21,56.11,.52]],
            [[0.0,10.0,2.0],[0.2,35.0,0.8],[0.0,10.0,1.7],[0.2,35.0,0.8]],
            [[0.0,10.0,1.3],[0.02,13.0,1.3],[0.04,58.39,0.88]],
            [[0.0,5.0,3.0],[0.02,10.0,2.0],[0.2,100.0,0.4],[0.05,15.0,1.6],[0.07,25.0,1.0]]]
# p_starts is a list of lists of lists. The first list is for each simulation, the second list is for each threshold, 
# and the third list is for the initial guesses for the fit parameters. The order of the parameters is gamma, r0, a.
for sim, p0s in zip(sims_good,p_starts):
    thresholds = glob.glob(sim+'_*.01024_two_point.dat')
    for thresh, p0 in zip(thresholds,p0s):
        print(thresh)
        fig=plt.figure(figsize=(15,10))
        ax = plt.subplot(111)
        data = pickle.load(open(thresh,'rb'), encoding='latin1') 
        young = np.where(data['age']<.05)
        #bins = np.linspace(0, data['r_half'], np.floor(data['r_half']/0.25)+1)
        upper = np.min([2*data['r_half'],10.0])
        bins = np.linspace(0, upper, np.ceil(upper/0.5)+1)
        #bins = np.linspace(0, 10, 41)
        #bins = 10**np.linspace(0, np.ceil(np.log10(3*data['r_half'])), np.floor(3*data['r_half']/0.2)+1)
        print(data['r_half'])
        if len(young[0]) > 0:
            X = np.asarray([data['pos'][young][:,0],data['pos'][young][:,1]])
            #corr, dcorr = bootstrap_two_point(X.T, bins, Nbootstrap=10)
            #ax.errorbar(bins[:-1],1+corr, dcorr)
            corr = two_point(X.T, bins)
            width = (bins[1] - bins[0])
            center = bins[1:] - width / 2.
            ax.plot(center, 1+corr)
            try:
                valid = ~np.isnan(corr)
                #popt, pcov = curve_fit(power_law, center[valid], 1+corr[valid])
                
                spopt, spcov = curve_fit(schechter, center[valid], 1+corr[valid], p0=p0)#[0.05,30.0,2.0])
            
                #valid = ~np.isnan(np.log10(corr))
                #log_popt, log_pcov = curve_fit(log_power_law, np.log10(center[valid]), np.log10(1+corr)[valid])
                
                #log_spopt, log_spcov = curve_fit(log_schechter, np.log10(center[valid]), np.log10(1+corr)[valid])
                
                #print('log fit not working')
                #print(thresh)
                
                #ax.plot(center, power_law(center, *popt), 'g--', label=r'fit: $\gamma=%.2f, r_{0}=%.2f, a=%.2f$' % tuple(popt))
                #ax.plot(center, 10**log_power_law(np.log10(center), *log_popt), 'r', label=r'fit: $\gamma=%.2f, \log(r_{0})=%.2f, \log(a)=%.2f$' % tuple(log_popt))
                #ax.plot(center, 10**log_schechter(np.log10(center), *log_spopt), 'g--', label=r'fit: $\gamma=%.2f, \log(r_{0})=%.2f, \log(a)=%.2f$' % tuple(log_spopt))
                
                ax.plot(center, schechter(center, *spopt), 'b--', label=r'fit: $\gamma=%.2f, r_{0}=%.2f, a=%.2f$' % tuple(spopt))
                
                f = open('./plots/'+thresh+'_0.05Gyr_plot.dat','wb')
                p.dump({'corr':corr,'bins':center,'popt':spopt,'pcov':spcov},f)
                f.close()
                
                ax.set_xscale('log')
                ax.set_yscale('log')
                ax.legend(loc=0, fontsize=20)
                ax.set_xlabel(r'$r$ [kpc]')
                ax.set_ylabel(r'$1+\xi\left(r\right)$')
                plt.savefig('./plots/'+thresh[:-4]+'.pdf',bbox_inches='tight')
            except:
                pass

        else:
            print('no young stars found!')
        

In [ ]:
# all redshift snaps below z=0.5
from scipy.optimize import curve_fit
sims = ['g7.05e09','g1.09e10','g1.18e10','g1.23e10','g2.34e10','g2.64e10','g4.27e10','g4.99e10','g1.37e11','g1.57e11','g1.59e11','g2.19e11','g3.21e11','g3.49e11','g3.55e11','g3.71e11','g7.08e11','g7.55e11','g7.66e11','g8.26e11']
snaps = ['.01024','.00992','.00960','.00928','.00896','.00864','.00832','.00800','.00768','.00736','.00704','.00672','.00640']

for sim in sims:
    for snap in snaps:
        thresholds = glob.glob(sim+'_*'+snap+'_two_point.dat')
        print(thresholds)
        for thresh in thresholds:
            fig=plt.figure(figsize=(15,10))
            ax = plt.subplot(111)
            data = p.load(open(thresh,'rb'), encoding='latin1') 
            young = np.where(data['age']<.05)
            
            upper = np.min([data['r_half'],5.])
            bins = np.linspace(0, upper, np.ceil(upper/0.3)+1)
        
            if len(young[0]) > 0:
                X = np.asarray([data['pos'][young][:,0],data['pos'][young][:,1]])
            
                corr = two_point(X.T, bins)
                width = (bins[1] - bins[0])
                center = bins[1:] - width / 2.
                ax.plot(center, 1+corr)
                try:
                    valid = ~np.isnan(corr)
                
                    spopt, spcov = curve_fit(schechter, center[valid], 1+corr[valid])#[0.05,30.0,2.0])
            
                    ax.plot(center, schechter(center, *spopt), 'b--', label=r'fit: $\gamma=%.2f, r_{0}=%.2f, a=%.2f$' % tuple(spopt))
                
                    f = open('./plots/'+thresh+'_0.05Gyr_plot.dat','wb')
                    p.dump({'corr':corr,'bins':center,'popt':spopt,'pcov':spcov},f)
                    f.close()
                
                    ax.set_xscale('log')
                    ax.set_yscale('log')
                    ax.legend(loc=0, fontsize=20)
                    ax.set_xlabel(r'$r$ [kpc]')
                    ax.set_ylabel(r'$1+\xi\left(r\right)$')
                    plt.savefig('./plots/'+thresh[:-4]+'.pdf',bbox_inches='tight')
                except:
                    pass

            else:
                print('no young stars found!')
        

In [ ]:
from scipy.optimize import curve_fit
sims_good = ['g2.64e10']#,'g8.26e11']

for sim in sims_good:
    thresholds = glob.glob(sim+'_*.01024_two_point.dat')
    for thresh in thresholds:
        n_temp = thresh.split('_')[1]
        e_temp = thresh.split('_')[2]
        n = n_temp[1:]
        e = e_temp[1:5]
        
        if n=='0.1' and e=='0.04':
            print(thresh)
            fig=plt.figure(figsize=(15,10))
            ax = plt.subplot(111)
            data = p.load(open(thresh,'rb'), encoding='latin1') 
            young = np.where(data['age']<.05)
            upper = np.min([data['r_half'],10.])
            bins = np.linspace(0, upper, np.ceil(upper/0.25)+1)
        
            print(data['r_half'])
            if len(young[0]) > 0:
                X = np.asarray([data['pos'][young][:,0],data['pos'][young][:,1]])
            
                corr = two_point(X.T, bins)
                width = (bins[1] - bins[0])
                center = bins[1:] - width / 2.
                ax.plot(center, 1+corr)
            
                valid = ~np.isnan(corr)
            
                spopt, spcov = curve_fit(schechter, center[valid], 1+corr[valid])#[0.05,30.0,2.0])
            
                ax.plot(center, schechter(center, *spopt), 'b--', label=r'fit: $\gamma=%.2f, r_{0}=%.2f, a=%.2f$' % tuple(spopt))
            
                f = open('./plots/'+thresh+'_0.05Gyr_plot.dat','wb')
                p.dump({'corr':corr,'bins':center,'popt':spopt,'pcov':spcov},f)
                f.close()
            
                ax.set_xscale('log')
                ax.set_yscale('log')
                ax.legend(loc=0, fontsize=20)
                ax.set_xlabel(r'$r$ [kpc]')
                ax.set_ylabel(r'$1+\xi\left(r\right)$')
                plt.savefig('./plots/'+thresh[:-4]+'.pdf',bbox_inches='tight')
            else:
                print('no young stars found!')
        

In [ ]:
from scipy.optimize import curve_fit
sims_good = ['g7.55e11']#,'g8.26e11']

p_starts = [[[0.0,10.0,2.0],[0.2,35.0,0.8],[0.0,10.0,1.7],[0.2,35.0,0.8]]]
            #[[0.0,5.0,3.0],[0.02,10.0,2.0],[0.2,100.0,0.4],[0.05,15.0,1.6],[0.07,25.0,1.0]]]


for sim, p0s in zip(sims_good,p_starts):
    thresholds = glob.glob(sim+'_*.01024_two_point.dat')
    for thresh, p0 in zip(thresholds,p0s):
        print(thresh)
        fig=plt.figure(figsize=(15,10))
        ax = plt.subplot(111)
        data = pickle.load(open(thresh,'rb'), encoding='latin1') 
        young = np.where(data['age']<.05)
        #bins = np.linspace(0, data['r_half'], np.floor(data['r_half']/0.25)+1)
        upper = np.min([2*data['r_half'],4.])
        bins = np.linspace(0, upper, np.ceil(upper/0.5)+1)
        #bins = np.linspace(0, 10, 41)
        #bins = 10**np.linspace(0, np.ceil(np.log10(3*data['r_half'])), np.floor(3*data['r_half']/0.2)+1)
        print(data['r_half'])
        if len(young[0]) > 0:
            X = np.asarray([data['pos'][young][:,0],data['pos'][young][:,1]])
            #corr, dcorr = bootstrap_two_point(X.T, bins, Nbootstrap=10)
            #ax.errorbar(bins[:-1],1+corr, dcorr)
            corr = two_point(X.T, bins)
            width = (bins[1] - bins[0])
            center = bins[1:] - width / 2.
            ax.plot(center, 1+corr)
            #try:
            valid = ~np.isnan(corr)
                #popt, pcov = curve_fit(power_law, center[valid], 1+corr[valid])
                
            spopt, spcov = curve_fit(schechter, center[valid], 1+corr[valid], p0=p0)#[0.05,30.0,2.0])
            
                #valid = ~np.isnan(np.log10(corr))
                #log_popt, log_pcov = curve_fit(log_power_law, np.log10(center[valid]), np.log10(1+corr)[valid])
                
                #log_spopt, log_spcov = curve_fit(log_schechter, np.log10(center[valid]), np.log10(1+corr)[valid])
                
                #print('log fit not working')
                #print(thresh)
                
                #ax.plot(center, power_law(center, *popt), 'g--', label=r'fit: $\gamma=%.2f, r_{0}=%.2f, a=%.2f$' % tuple(popt))
                #ax.plot(center, 10**log_power_law(np.log10(center), *log_popt), 'r', label=r'fit: $\gamma=%.2f, \log(r_{0})=%.2f, \log(a)=%.2f$' % tuple(log_popt))
                #ax.plot(center, 10**log_schechter(np.log10(center), *log_spopt), 'g--', label=r'fit: $\gamma=%.2f, \log(r_{0})=%.2f, \log(a)=%.2f$' % tuple(log_spopt))
                
            ax.plot(center, schechter(center, *spopt), 'b--', label=r'fit: $\gamma=%.2f, r_{0}=%.2f, a=%.2f$' % tuple(spopt))
                
            ax.plot(center, schechter(center, *p0), 'r--', label=r'fit: $\gamma=%.2f, r_{0}=%.2f, a=%.2f$' % tuple(p0))
                
            #except:
                #print('shit')
            f = open('./plots/'+thresh+'_0.05Gyr_plot.dat','wb')
            p.dump({'corr':corr,'bins':center,'popt':spopt,'pcov':spcov},f)
            f.close()
            
            ax.set_xscale('log')
            ax.set_yscale('log')
            ax.legend(loc=0, fontsize=20)
            ax.set_xlabel(r'$r$ [kpc]')
            ax.set_ylabel(r'$1+\xi\left(r\right)$')
            plt.savefig('./plots/'+thresh[:-4]+'.pdf',bbox_inches='tight')
        else:
            print('no young stars found!')
        

In [ ]:
data10 = pickle.load(open('g3.49e11_n10.0_e0.13.01024_two_point.dat','rb'), encoding='latin1')

In [ ]:
data1 = pickle.load(open('g3.49e11_n1.0_e0.13.01024_two_point.dat','rb'), encoding='latin1')

In [ ]:
data01 = pickle.load(open('g3.49e11_n0.1_e0.04.01024_two_point.dat','rb'), encoding='latin1')

In [ ]:
young10 = np.where(data10['age']<.025)
young1 = np.where(data1['age']<.025)
young01 = np.where(data01['age']<.025)

In [ ]:
from astroML.correlation import two_point
from astroML.correlation import bootstrap_two_point

In [ ]:
X10 = np.asarray([data10['pos'][young10][:,0],data10['pos'][young10][:,1]])

In [ ]:
corr10 = two_point(X10.T, bins)

In [ ]:
f = open('./g3.49e11_n10.0_e0.13.01024_0.025Gyr_plot.dat','wb')
p.dump({'corr':corr10,'bins':bins},f)
f.close()

In [ ]:
X1 = np.asarray([data1['pos'][young1][:,0],data1['pos'][young1][:,1]])

In [ ]:
corr1 = two_point(X1.T, bins)

In [ ]:
import pickle as p
f = open('./g3.49e11_n1.0_e0.13.01024_0.025Gyr_plot.dat','wb')
p.dump({'corr':corr1,'bins':bins},f)
f.close()

In [ ]:
X01 = np.asarray([data01['pos'][young01][:,0],data01['pos'][young01][:,1]])

In [ ]:
corr01 = two_point(X01.T, bins)

In [ ]:
f = open('./g3.49e11_n0.1_e0.13.01024_0.025Gyr_plot.dat','wb')
p.dump({'corr':corr01,'bins':bins},f)
f.close()

In [ ]:
data = pickle.load(open('./g3.49e11_n10.0_e0.13.01024_plot.dat','rb'))
corr10 = data['corr']

In [ ]:
plt.plot(bins[:-1],1+corr10,label=r'$n_{\rm th}=10.0$')
plt.plot(bins[:-1],1+corr1,label=r'$n_{\rm th}=1.0$')
plt.plot(bins[:-1],1+corr01,label=r'$n_{\rm th}=0.1$')
#plt.errorbar(bins[:-1],1+corr, dcorr)
plt.xlim([0.4,10])
plt.ylim([0.8,2])
plt.xscale('log')
#plt.yscale('log')
plt.xlabel(r'$r$ [kpc]')
plt.ylabel(r'$1+\omega\left(\theta\right)$')
plt.legend(loc=0,fontsize=20)

In [ ]:
plt.figure(figsize=(15,10))
ages = glob.glob('./g3.49e11_n0.1_e0.13.01024_*Gyr_plot.dat')
ages1 = glob.glob('./g3.49e11_n1.0_e0.13.01024_*Gyr_plot.dat')
ages10 = glob.glob('./g3.49e11_n10.0_e0.13.01024_*Gyr_plot.dat')

c = ['orange','teal','darkmagenta','k']
i = 0
for age, age1, age10 in zip(ages,ages1,ages10):
    data = pickle.load(open(age,'rb'))
    label = age.split('_')[-2]
    if label not in ['0.025Gyr','1Gyr','2Gyr']:
        plt.plot(bins[:-1],1+data['corr'],ls='dotted',color=c[i],label=label)
    
    data = pickle.load(open(age1,'rb'))
    label = age1.split('_')[-2]
    if label not in ['0.025Gyr','1Gyr','2Gyr']:
        plt.plot(bins[:-1],1+data['corr'],ls='dashed',color=c[i],label=label)
    
    data = pickle.load(open(age10,'rb'))
    label = age10.split('_')[-2]
    if label not in ['0.025Gyr','1Gyr','2Gyr']:
        plt.plot(bins[:-1],1+data['corr'],color=c[i],label=label)
        i+=1
    
plt.xlim([0.4,10])
plt.ylim([0.8,3.5])
plt.xscale('log')
plt.yscale('log')
plt.xlabel(r'$r$ [kpc]')
plt.ylabel(r'$1+\omega\left(\theta\right)$')
plt.legend(loc=0,fontsize=20,ncol=4)
plt.savefig('g3.49e11_two_point.pdf',bbox_inches='tight')

# fit parameters vs. stellar mass

In [ ]:
sims = ['g7.05e09','g2.34e10','g2.64e10','g4.27e10','g4.99e10','g1.37e11','g1.57e11','g1.59e11','g2.19e11','g3.21e11','g3.49e11','g3.55e11','g3.71e11','g7.08e11','g7.55e11','g7.66e11','g8.26e11']
#sims = ['g1.37e11','g1.57e11','g1.59e11','g3.49e11','g3.55e11','g3.71e11','g7.08e11','g7.55e11','g7.66e11','g8.26e11'] #,'g2.19e11','g3.21e11'
snaps = ['.01024','.00992','.00960','.00928','.00896','.00864','.00832','.00800','.00768','.00736','.00704','.00672','.00640']
#g1.09e10, g1.18e10, g1.23e10

In [ ]:
gamma_obs = [0.23,0.26,0.23,0.14,0.21]
gamma_obs_err = [0.03,0.02,0.02,0.02,0.02]

r0_obs = [482.4399935741676, 572.1789188080746, 164.24744712944786, 626.5544092341999, 4270.361694249362, 2504.047318383617]
r0_obs_err_h = [545.398856363284,731.2631562681235,196.3227769347292,732.4165398498108,5161.849764912015,2707.3363085347796]
r0_obs_err_l = [412.70901013076275,423.41188487159724,132.88783346135827,524.1719974133647,3304.304105696204,2316.022931077103]
mstar_obs = [239311342.07536945,1884883516.1732612,3140034080.586631,5005592188.793732,10941311681.503845,26919330369.703316]

sfr_obs = [0.054380664652567745, 0.3081570996978851, 0.5256797583081569, 0.8519637462235647, 3.6616314199395767, 5.673716012084592]   

In [ ]:
#
fig=plt.figure(figsize=(15,10))
ax = plt.subplot(111)
    
for sim in sims:
    for snap in snaps:
        thresholds = glob.glob(sim+'_*'+snap+'_two_point.dat')
        for thresh in thresholds:
        
            n_temp = thresh.split('_')[1]
            e_temp = thresh.split('_')[2]
            n = n_temp[1:]
            e = e_temp[1:5]
            try:
                data = p.load(open(thresh,'rb'), encoding='latin1') 
                mstar = data['mstar']
        
                data = p.load(open('./plots/'+thresh+'_0.05Gyr_plot.dat','rb'))
                popt = data['popt']
                pcov = data['pcov']
        
                gamma = popt[0]
                r0 = popt[1]
                a = popt[2]

                gamma_err = np.sqrt( pcov[0][0] )
                r0_err = np.sqrt( pcov[1][1] )
                a_err = np.sqrt( pcov[2][2] )
        
                if n == '10.0':
                    ax.errorbar(mstar, gamma, yerr=gamma_err, c='r', marker='s')#=50)
                elif n == '1.0':
                    ax.errorbar(mstar, gamma, yerr=gamma_err, c='k', marker='^')
                elif n == '0.1':
                    if e == '0.04':
                        ax.errorbar(mstar, gamma, yerr=gamma_err, c='b', marker='o')
            except:
                pass
        
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='r', marker='s', label=r'$n=10.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='k', marker='^', label=r'$n=1.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='b', marker='o', label=r'$n=0.1$')

ax.fill_between([3e9,7e10],y1=0.12,y2=0.28, color='lightgray', label='LEGUS (Grasha+2017)')

ax.set_xlim(1e8,7e10)
ax.set_ylim(-0.1,0.5)
ax.set_xscale('log')
#ax.set_yscale('log')
ax.legend(loc=0, fontsize=20)
ax.set_xlabel(r'$M_{\star}$ [$M_{\odot}$]')
ax.set_ylabel(r'powerlaw index $\gamma$')
plt.savefig('./plots/gamma_mstar.pdf',bbox_inches='tight')

        

In [ ]:
#
fig=plt.figure(figsize=(15,10))
ax = plt.subplot(111)
    
for sim in sims:
    for snap in snaps:
        thresholds = glob.glob(sim+'_*'+snap+'_two_point.dat')

        for thresh in thresholds:
            try:
                n_temp = thresh.split('_')[1]
                e_temp = thresh.split('_')[2]
                n = n_temp[1:]
                e = e_temp[1:5]
        
                data = p.load(open(thresh,'rb'), encoding='latin1') 
                mstar = data['r_half']
        
                data = p.load(open('./plots/'+thresh+'_0.05Gyr_plot.dat','rb'))
                popt = data['popt']
                pcov = data['pcov']
        
                gamma = popt[0]
                r0 = popt[1]
                a = popt[2]

                gamma_err = np.sqrt( pcov[0][0] )
                r0_err = np.sqrt( pcov[1][1] )
                a_err = np.sqrt( pcov[2][2] )
        
                if n == '10.0':
                    ax.errorbar(mstar, gamma, yerr=gamma_err, c='r', marker='s')
                elif n == '1.0':
                    ax.errorbar(mstar, gamma, yerr=gamma_err, c='k', marker='^')
                elif n == '0.1':
                    if e == '0.04':
                        ax.errorbar(mstar, gamma, yerr=gamma_err, c='b', marker='o')
            except:
                pass
            
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='r', marker='s', label=r'$n=10.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='k', marker='^', label=r'$n=1.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='b', marker='o', label=r'$n=0.1$')

ax.fill_between([1.9,1.5e1],y1=0.12,y2=0.28, color='lightgray', label='LEGUS (Grasha+2017)')

ax.set_xlim(1.9,1.5e1)
ax.set_ylim(-0.1,0.5)
ax.set_xscale('log')
#ax.set_yscale('log')
ax.legend(loc=0, fontsize=20)
ax.set_xlabel(r'$R_{\rm half}$ [kpc]')
ax.set_ylabel(r'powerlaw index $\gamma$')
plt.savefig('./plots/gamma_rhalf.pdf',bbox_inches='tight')

        

In [ ]:
#
fig=plt.figure(figsize=(15,10))
ax = plt.subplot(111)
    
for sim in sims:
    for snap in snaps:
        thresholds = glob.glob(sim+'_*'+snap+'_two_point.dat')
        
        for thresh in thresholds:
            try:
                n_temp = thresh.split('_')[1]
                e_temp = thresh.split('_')[2]
                n = n_temp[1:]
                e = e_temp[1:5]
        
                data = p.load(open(thresh,'rb'), encoding='latin1') 
                mstar = data['mstar']
        
                data = p.load(open('./plots/'+thresh+'_0.05Gyr_plot.dat','rb'))
                popt = data['popt']
                pcov = data['pcov']
        
                gamma = popt[0]
                r0 = popt[1]*1000.
                a = popt[2]

                gamma_err = np.sqrt( pcov[0][0] )
                r0_err = np.sqrt( pcov[1][1] )*1000.
                a_err = np.sqrt( pcov[2][2] )

                if n == '10.0':
                    ax.errorbar(mstar, r0, yerr=r0_err, c='r', marker='s')#=50)
                elif n == '1.0':
                    ax.errorbar(mstar, r0, yerr=r0_err, c='k', marker='^')
                elif n == '0.1':
                    if e == '0.04':
                        ax.errorbar(mstar, r0, yerr=r0_err, c='b', marker='o')
            except:
                pass
        
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='r', marker='s', label=r'$n=10.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='k', marker='^', label=r'$n=1.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='b', marker='o', label=r'$n=0.1$')

ax.errorbar(mstar_obs,r0_obs,yerr=[r0_obs_err_l,r0_obs_err_h],c='lightgray',fmt='*',markersize=30,label='LEGUS (Grasha+2017)')

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim(1e9,7.05e10)
ax.set_ylim(5,1e5)
ax.legend(loc=0, fontsize=20)
ax.set_xlabel(r'$M_{\star}$ [$M_{\odot}$]')
ax.set_ylabel(r'correlation length $r_{0}$ [pc]')
plt.savefig('./plots/r0_mstar.pdf',bbox_inches='tight')

        

In [ ]:
#
fig=plt.figure(figsize=(15,10))
ax = plt.subplot(111)
    
for sim in sims:
    for snap in snaps:
        thresholds = glob.glob(sim+'_*'+snap+'_two_point.dat')
        
        for thresh in thresholds:
            try:
                n_temp = thresh.split('_')[1]
                e_temp = thresh.split('_')[2]
                n = n_temp[1:]
                e = e_temp[1:5]
        
                data = p.load(open('./plots/'+thresh+'_0.05Gyr_plot.dat','rb'))
                popt = data['popt']
                pcov = data['pcov']
        
                gamma = popt[0]
                r0 = popt[1]
                a = popt[2]

                gamma_err = np.sqrt( pcov[0][0] )
                r0_err = np.sqrt( pcov[1][1] )
                a_err = np.sqrt( pcov[2][2] )

                if n == '10.0':
                    ax.errorbar(gamma, r0, xerr=gamma_err, yerr=r0_err, c='r', marker='s')#=50)
                elif n == '1.0':
                    ax.errorbar(gamma, r0, xerr=gamma_err, yerr=r0_err, c='k', marker='^')
                elif n == '0.1':
                    if e == '0.04':
                        ax.errorbar(gamma, r0, xerr=gamma_err, yerr=r0_err, c='b', marker='o')
            except:
                pass
        
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='r', marker='s', label=r'$n=10.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='k', marker='^', label=r'$n=1.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='b', marker='o', label=r'$n=0.1$')

ax.fill_between([-0.1,0.5],y1=np.mean(r0_obs)-np.std(r0_obs)/np.sqrt(6),y2=np.mean(r0_obs)+np.std(r0_obs)/np.sqrt(6), color='lightgray', label='LEGUS (Grasha+2017)')
ax.fill_between([0.12,0.28],y1=3,y2=2.5e3, color='lightgray')

ax.set_xlim(-0.1,0.5)
ax.set_ylim(1,1e4)
#ax.set_xscale('log')
ax.set_yscale('log')
ax.legend(loc=0, fontsize=20)
ax.set_xlabel(r'powerlaw index $\gamma$')
ax.set_ylabel(r'correlation length $r_{0}$')
plt.savefig('./plots/gamma_vs_r0.pdf',bbox_inches='tight')



In [ ]:
# paper plot
fig=plt.figure(figsize=(32,10))
gs = gridspec.GridSpec(1, 2)
gs.update(wspace=.175, hspace=0.)  # set the spacing between axes.
ax1 = plt.subplot(gs[0])
ax1.set_ylabel(r'normalized counts')
ax1.set_xlabel(r'powerlaw index $\gamma$')
ax2 = plt.subplot(gs[1])
ax2.set_ylabel(r'normalized counts')
ax2.set_xlabel(r'$\log(r_{0}/\rm{pc}$)')

bins = 'auto'
hist, bin_edg = np.histogram(np.asarray(gamma_10), normed=True, range=(-.2,.4), bins=bins)
ax1.step(bin_edg[:-1], hist, c='darkmagenta',lw=7, label=r'$n=10.0$')#, where='mid')
ax1.plot([np.median(gamma_10),np.median(gamma_10)],[0,14], c='darkmagenta',zorder=-1)

hist, bin_edg = np.histogram(np.asarray(gamma_1), normed=True, range=(-.2,.4), bins=bins)
ax1.step(bin_edg[:-1], hist, c='k',lw=7,label=r'$n=1.0$')#, where='mid')
ax1.plot([np.median(gamma_1),np.median(gamma_1)],[0,14], c='k',zorder=-1)

hist, bin_edg = np.histogram(np.asarray(gamma_01), normed=True, range=(-.2,.4), bins=bins)
ax1.step(bin_edg[:-1], hist, c='c',lw=7,label=r'$n=0.1$')#, where='mid')
ax1.plot([np.median(gamma_01),np.median(gamma_01)],[0,14], c='c',zorder=-1)

ax1.fill_between([0.12,0.28],y1=0.0,y2=14.5, color='lightgray', label='LEGUS (Grasha+2017)', zorder=-1, alpha=.5)

#ax1.set_ylim(-0.25,14.5)

# correlation length
hist, bin_edg = np.histogram(np.asarray(np.log10(r0_10)), normed=True, range=(2.5,5), bins=bins)
ax2.step(bin_edg[:-1], hist, c='darkmagenta',lw=7)#, where='mid')
ax2.plot([np.median(np.log10(r0_10)),np.median(np.log10(r0_10))],[0,2.2], c='darkmagenta',zorder=-1)

hist, bin_edg = np.histogram(np.asarray(np.log10(r0_1)), normed=True, range=(2.5,5), bins=bins)
ax2.step(bin_edg[:-1], hist, c='k',lw=7)#, where='mid')
ax2.plot([np.median(np.log10(r0_1)),np.median(np.log10(r0_1))],[0,2.2], c='k',zorder=-1)

hist, bin_edg = np.histogram(np.asarray(np.log10(r0_01)), normed=True, range=(2.5,5), bins=bins)
ax2.step(bin_edg[:-1], hist, c='c',lw=7)#, where='mid')
ax2.plot([np.median(np.log10(r0_01)),np.median(np.log10(r0_01))],[0,2.2], c='c',zorder=-1)

ax2.fill_between([np.log10(np.mean(r0_obs)-np.std(r0_obs)/np.sqrt(6)),np.log10(np.mean(r0_obs)+np.std(r0_obs)/np.sqrt(6))],y1=0.0,y2=2.75, color='lightgray', label='LEGUS (Grasha+2017)', zorder=-1, alpha=.5)

ax1.legend(loc=0)#, fontsize=25)

plt.savefig('distribution.pdf', bbox_inches='tight')

In [ ]:
#
from scipy.stats import binned_statistic as bs
from scipy import stats

bins = 5
range = None #(0,10)
#mstar_10 = np.log10(mstar_10)
#mstar_1 = np.log10(mstar_1)
#mstar_01 = np.log10(mstar_01)

fig=plt.figure(figsize=(10,13.2))

gs = gridspec.GridSpec(2, 1)
gs.update(wspace=.0, hspace=0.)  # set the spacing between axes.

ax = plt.subplot(gs[0])
x = np.linspace(0.03,60,200)

#inverse relative error

ax.scatter(np.asarray(sfr_5_10)/np.asarray(sfr_200_10), np.asarray(r0_10), c='darkmagenta', marker='s',alpha=.5,edgecolors='none')#=50)

#inverse relative error n=1

ax.scatter(np.asarray(sfr_5_1)/np.asarray(sfr_200_1), np.asarray(r0_1), c='k', marker='^',alpha=.5,edgecolors='none')#=50)

#inverse relative error n=0.1

ax.scatter(np.asarray(sfr_5_01)/np.asarray(sfr_200_01), np.asarray(r0_01), c='c', marker='o',alpha=.5, edgecolors='none')#, marker='o',  facecolors='none',alpha=.5)#=50)

log_sfr_10 = np.log10(np.asarray(sfr_5_10)/np.asarray(sfr_200_10))
tmp = np.asarray(sfr_5_10)/np.asarray(sfr_200_10) == 0.
log_sfr_10[tmp] = 1e-8
xd = log_sfr_10
yd = np.log10(np.asarray(r0_10))
slope_10, intercept_10, r_value_10, p_value_10, std_err = stats.linregress(xd,yd)

plt.plot(x,10**(slope_10*np.log10(x)+intercept_10), c='darkmagenta',lw=6,label=r'p-value=%.2f'%p_value_10)


log_sfr_1 = np.log10(np.asarray(sfr_5_1)/np.asarray(sfr_200_1))
tmp = np.asarray(sfr_5_1)/np.asarray(sfr_200_1) == 0.
log_sfr_1[tmp] = 1e-8
xd = log_sfr_1
yd = np.log10(np.asarray(r0_1))
slope_1, intercept_1, r_value_1, p_value_1, std_err = stats.linregress(xd,yd)

#plt.plot(x,10**(slope*np.log10(x)+intercept), c='k',lw=6)
plt.plot(x,10**(slope_1*np.log10(x)+intercept_1), c='k',lw=6,label=r'p-value=%.2f'%p_value_1)


log_sfr_01 = np.log10(np.asarray(sfr_5_01)/np.asarray(sfr_200_01))
tmp = np.asarray(sfr_5_01)/np.asarray(sfr_200_01) == 0.
log_sfr_01[tmp] = 1e-8
xd = log_sfr_01
yd = np.log10(np.asarray(r0_01))
slope, intercept, r_value, p_value, std_err = stats.linregress(xd,yd)

#plt.plot(x,10**(slope*np.log10(x)+intercept), c='c',lw=6)
plt.plot(x,10**(slope*np.log10(x)+intercept), c='c',lw=6,label=r'p-value=%.2f'%p_value)

plt.plot([],[], c='w',lw=6,label=r'r-value=%.2f'%r_value_10)
plt.plot([],[], c='w',lw=6,label=r'r-value=%.2f'%r_value_1)
plt.plot([],[], c='w',lw=6,label=r'r-value=%.2f'%r_value)


#ax.fill_between([0.07,2e1],y1=np.mean(r0_obs)-np.std(r0_obs)/np.sqrt(6),y2=np.mean(r0_obs)+np.std(r0_obs)/np.sqrt(6), color='lightgray', label='LEGUS (Grasha+2017)', zorder=-1,alpha=.5)
 
#ax.fill_between(,y1=0.12,y2=0.28, color='lightgray', label='LEGUS (Grasha+2017)', zorder=-1, alpha=.5)

ax.scatter([], [], c='darkmagenta', marker='s', label=r'$n=10.0$')
ax.scatter([], [], c='k', marker='^', label=r'$n=1.0$')
ax.scatter([], [], c='c', marker='o', label=r'$n=0.1$')

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_ylim(2e2,1e6)
ax.set_xlim(0.03,60)
ax.legend(loc=0, fontsize=15,ncol=3)
#ax.set_xlabel(r'SFR(5 Myr)/SFR(200 Myr)')
ax.set_ylabel(r'correlation length $r_{0}$ [pc]')

#################################################################################

idx = np.where(np.asarray(gamma_1)>-.6)

ax = plt.subplot(gs[1])

ax.scatter(np.asarray(sfr_5_10)/np.asarray(sfr_200_10), np.asarray(gamma_10), c='darkmagenta', marker='s',alpha=.5, edgecolors='none')#=50)

#inverse relative error n=1

ax.scatter(np.asarray(sfr_5_1)[idx]/np.asarray(sfr_200_1)[idx], np.asarray(gamma_1)[idx], c='k', marker='^',alpha=.5, edgecolors='none')#=50)

#inverse relative error n=0.1

ax.scatter(np.asarray(sfr_5_01)/np.asarray(sfr_200_01), np.asarray(gamma_01), c='c', marker='o',alpha=.5, edgecolors='none')#=50)

log_sfr_10 = np.log10(np.asarray(sfr_5_10)/np.asarray(sfr_200_10))
tmp = np.asarray(sfr_5_10)/np.asarray(sfr_200_10) == 0.
log_sfr_10[tmp] = 1e-8
xd = log_sfr_10
yd = np.asarray(gamma_10)
slope_10, intercept_10, r_value_10, p_value_10, std_err = stats.linregress(xd,yd)

plt.plot(x,(slope_10*np.log10(x)+intercept_10), c='darkmagenta',lw=6,label=r'p-value=%.2f'%p_value_10)


log_sfr_1 = np.log10(np.asarray(sfr_5_1)/np.asarray(sfr_200_1))
tmp = np.asarray(sfr_5_1)/np.asarray(sfr_200_1) == 0.
log_sfr_1[tmp] = 1e-8
xd = log_sfr_1
yd = np.asarray(gamma_1)
slope_1, intercept_1, r_value_1, p_value_1, std_err = stats.linregress(xd,yd)

#plt.plot(x,10**(slope*np.log10(x)+intercept), c='k',lw=6)
plt.plot(x,(slope_1*np.log10(x)+intercept_1), c='k',lw=6,label=r'p-value=%.2f'%p_value_1)


log_sfr_01 = np.log10(np.asarray(sfr_5_01)/np.asarray(sfr_200_01))
tmp = np.asarray(sfr_5_01)/np.asarray(sfr_200_01) == 0.
log_sfr_01[tmp] = 1e-8
xd = log_sfr_01
yd = np.asarray(gamma_01)
slope, intercept, r_value, p_value, std_err = stats.linregress(xd,yd)

#plt.plot(x,10**(slope*np.log10(x)+intercept), c='c',lw=6)
plt.plot(x,(slope*np.log10(x)+intercept), c='c',lw=6,label=r'p-value=%.2f'%p_value)

plt.plot([],[], c='w',lw=6,label=r'r-value=%.2f'%r_value_10)
plt.plot([],[], c='w',lw=6,label=r'r-value=%.2f'%r_value_1)
plt.plot([],[], c='w',lw=6,label=r'r-value=%.2f'%r_value)


ax.scatter([], [], c='darkmagenta', marker='s', label=r'$n=10.0$')
ax.scatter([], [], c='k', marker='^', label=r'$n=1.0$')
ax.scatter([], [], c='c', marker='o', label=r'$n=0.1$')

#ax.fill_between([0.07,2e1],y1=0.12,y2=0.28, color='lightgray', label='LEGUS (Grasha+2017)', zorder=-1, alpha=.5)

ax.set_xscale('log')
#ax.set_yscale('log')
ax.set_ylim(-0.25,0.6)
ax.set_xlim(0.03,60)
ax.legend(loc=0, fontsize=15,ncol=3)
ax.set_xlabel(r'SFR(5 Myr)/SFR(200 Myr)')
ax.set_ylabel(r'powerlaw index $\gamma$')
plt.savefig('./plots/gamma_r0_sfr_ratio_unbinned.pdf',bbox_inches='tight')


In [ ]:
from scipy.optimize import curve_fit
sims_good = ['g3.55e11']#,'g8.26e11']
lim = [0.95,2.3]

fig=plt.figure(figsize=(22.5,5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1,1,1])
gs.update(wspace=.0, hspace=0.)  # set the spacing between axes.
ax01 = plt.subplot(gs[0])
ax01.set_ylabel(r'$1+\xi\left(r\right)$')
ax01.set_xlabel(r'$r$ [kpc]')
ax01.set_xscale('log')
ax01.set_yscale('log')
ax01.set_ylim(lim)

ax1 = plt.subplot(gs[1])
#ax1.set_xticks([-30,-15,0,15,30])
ax1.set_xlabel(r'$r$ [kpc]')
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_ylim(lim)


ax10 = plt.subplot(gs[2])
ax10.set_xlabel(r'$r$ [kpc]')
ax10.set_xscale('log')
ax10.set_yscale('log')
ax10.set_ylim(lim)


for sim in sims_good:
    thresholds = glob.glob(sim+'_*.00928_two_point.dat')
    for thresh, axis, c in zip([thresholds[0],thresholds[1],thresholds[-1]],[ax01,ax1,ax10],['c','k','darkmagenta']):
        
        n_temp = thresh.split('_')[1]
        e_temp = thresh.split('_')[2]
        n = n_temp[1:]
        e = e_temp[1:5]
        
        if n == '0.1' and e == '0.13':
            pass
        else:
            print(thresh)
            data = p.load(open(thresh,'rb'), encoding='latin1') 
            young = np.where(data['age']<.05)
            
            if n == '10.0':
                upper = np.min([2*data['r_half'],5.3])
                bins = np.linspace(0, upper, np.ceil(upper/0.25)+1)
            else:
                upper = np.min([2*data['r_half'],5.3])
                bins = np.linspace(0, upper, np.ceil(upper/0.25)+1)

            if len(young[0]) > 0:
                X = np.asarray([data['pos'][young][:,0],data['pos'][young][:,1]])
            
                corr = two_point(X.T, bins)
                width = (bins[1] - bins[0])
                center = bins[1:] - width / 2.
                axis.plot(center, 1+corr, c=c)
            
                valid = ~np.isnan(corr)
            
                spopt, spcov = curve_fit(schechter, center[valid], 1+corr[valid])
            
                err = np.sqrt(np.diag(spcov))
                
                axis.plot(center, schechter(center, *spopt), 'r', ls='dashed', label=r'fit: $\gamma=%.2f, r_{0}=%.2f, a=%.2f$' % tuple(spopt))
                axis.plot([], [], 'w--', label=r'     $\gamma_{\rm err}=%.2f, r_{\rm 0, err}=%.2f, a_{\rm err}=%.2f$' % tuple(np.asarray(err)))
               
                axis.legend(loc=0, fontsize=15)

            else:
                print('no young stars found!')

ax1.set_yticklabels([''],minor='minor')
ax10.set_yticklabels('',minor='minor')
ax1.set_yticklabels([''])
ax10.set_yticklabels('')
plt.savefig('./plots/example_fit.pdf', bbox_inches='tight')

# binned plots

In [ ]:
# error weighted average in stellar mass bins

gamma_10 = []
gamma_1 = []
gamma_01 = []

gamma_10_err = []
gamma_1_err = []
gamma_01_err = []

r0_10 = []
r0_1 = []
r0_01 = []

r0_10_err = []
r0_1_err = []
r0_01_err = []

mstar_10 = []
mstar_1 = []
mstar_01 = []

rhalf_10 = []
rhalf_1 = []
rhalf_01 = []

sfr_10 = []
sfr_1 = []
sfr_01 = []

sfr_200_10 = []
sfr_200_1 = []
sfr_200_01 = []

sfr_5_10 = []
sfr_5_1 = []
sfr_5_01 = []

sfr_1000_10 = []
sfr_1000_1 = []
sfr_1000_01 = []

for sim in sims:
    for snap in snaps:
        thresholds = glob.glob(sim+'_*'+snap+'_two_point.dat')
        for thresh in thresholds:
        
            n_temp = thresh.split('_')[1]
            e_temp = thresh.split('_')[2]
            n = n_temp[1:]
            e = e_temp[1:5]
            try:
                data = p.load(open(thresh,'rb'), encoding='latin1') 
                mstar = data['mstar']
                rhalf = data['r_half']
                age = data['age']
                massform = data['massform']
        
                young = np.where(age < 1.0)
                young_stars = np.asarray(massform)[young]
                sfr_1000 = np.sum(young_stars) / 1000000000. # msun / yr
                
                young = np.where(age < 0.2)
                young_stars = np.asarray(massform)[young]
                sfr_200 = np.sum(young_stars) / 200000000. # msun / yr
                
                young = np.where(age < 0.1)
                young_stars = np.asarray(massform)[young]
                sfr = np.sum(young_stars) / 100000000. # msun / yr
            
                young = np.where(age < 0.005)
                young_stars = np.asarray(massform)[young]
                sfr_5 = np.sum(young_stars) / 5000000. # msun / yr
                
                data = p.load(open('./plots/'+thresh+'_0.05Gyr_plot.dat','rb'))
                popt = data['popt']
                pcov = data['pcov']
        
                gamma = popt[0]
                r0 = popt[1] * 1000.
                a = popt[2]

                gamma_err = np.sqrt( pcov[0][0] )
                r0_err = np.sqrt( pcov[1][1] ) * 1000.
                a_err = np.sqrt( pcov[2][2] )
        
                if n == '10.0':
                    gamma_10.append(gamma)
                    gamma_10_err.append(gamma_err)
                    r0_10.append(r0)
                    r0_10_err.append(r0_err)
                    mstar_10.append(mstar)
                    rhalf_10.append(rhalf)
                    sfr_10.append(sfr)
                    sfr_1000_10.append(sfr_1000)
                    sfr_5_10.append(sfr_5)
                    sfr_200_10.append(sfr_200)
                    
                elif n == '1.0':
                    gamma_1.append(gamma)
                    gamma_1_err.append(gamma_err)
                    r0_1.append(r0)
                    r0_1_err.append(r0_err)
                    mstar_1.append(mstar)
                    rhalf_1.append(rhalf)
                    sfr_1.append(sfr)
                    sfr_1000_1.append(sfr_1000)
                    sfr_5_1.append(sfr_5)
                    sfr_200_1.append(sfr_200)
                    
                elif n == '0.1':
                    if e == '0.04':
                        gamma_01.append(gamma)
                        gamma_01_err.append(gamma_err)
                        r0_01.append(r0)
                        r0_01_err.append(r0_err)
                        mstar_01.append(mstar)
                        rhalf_01.append(rhalf)
                        sfr_01.append(sfr)
                        sfr_1000_01.append(sfr_1000)
                        sfr_5_01.append(sfr_5)
                        sfr_200_01.append(sfr_200)
                        if r0 < 3e3:
                            print(thresh)
                        
            except:
                pass

In [ ]:
#
from scipy.stats import binned_statistic as bs

bins = 5
range = None #(0,10)
#mstar_10 = np.log10(mstar_10)
#mstar_1 = np.log10(mstar_1)
#mstar_01 = np.log10(mstar_01)

fig=plt.figure(figsize=(15,10))
ax = plt.subplot(111)

#inverse relative error
weights = np.abs(np.asarray(r0_10)/np.asarray(r0_10_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(np.asarray(sfr_5_10)/np.asarray(sfr_10), np.asarray(r0_10)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(np.asarray(sfr_5_10)/np.asarray(sfr_10), weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(np.asarray(sfr_5_10)/np.asarray(sfr_10), np.asarray(r0_10)*weights*np.asarray(r0_10)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(np.asarray(sfr_5_10)/np.asarray(sfr_10), weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(np.asarray(sfr_5_10)/np.asarray(sfr_10), np.asarray(r0_10), statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax.errorbar(bin_centers, hist, yerr=std/np.sqrt(n), c='r', marker='s')#=50)

#inverse relative error n=1
weights = np.abs(np.asarray(r0_1)/np.asarray(r0_1_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(np.asarray(sfr_5_1)/np.asarray(sfr_1), np.asarray(r0_1)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(np.asarray(sfr_5_1)/np.asarray(sfr_1), weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(np.asarray(sfr_5_1)/np.asarray(sfr_1), np.asarray(r0_1)*weights*np.asarray(r0_1)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(np.asarray(sfr_5_1)/np.asarray(sfr_1), weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(np.asarray(sfr_5_1)/np.asarray(sfr_1), np.asarray(r0_1), statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax.errorbar(bin_centers, hist, yerr=std/np.sqrt(n), c='k', marker='^')#=50)

#inverse relative error n=0.1
weights = np.abs(np.asarray(r0_01)/np.asarray(r0_01_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(np.asarray(sfr_5_01)/np.asarray(sfr_01), np.asarray(r0_01)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(np.asarray(sfr_5_01)/np.asarray(sfr_01), weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(np.asarray(sfr_5_01)/np.asarray(sfr_01), np.asarray(r0_01)*weights*np.asarray(r0_01)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(np.asarray(sfr_5_01)/np.asarray(sfr_01), weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(np.asarray(sfr_5_01)/np.asarray(sfr_01), np.asarray(r0_01), statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax.errorbar(bin_centers, hist, yerr=std/np.sqrt(n), c='b', marker='o')#=50)
    
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='r', marker='s', label=r'$n=10.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='k', marker='^', label=r'$n=1.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='b', marker='o', label=r'$n=0.1$')

#ax.set_xscale('log')
ax.set_yscale('log')
ax.set_ylim(4e2,5e4)
ax.legend(loc=0, fontsize=20)
ax.set_xlabel(r'SFR(5 Myr)/SFR(100 Myr)')
ax.set_ylabel(r'correlation length $r_{0}$ [pc]')
plt.savefig('./plots/r0_sfr_ratio.pdf',bbox_inches='tight')



In [ ]:
#
from scipy.stats import binned_statistic as bs

bins = 5
range = None #(0,10)
#mstar_10 = np.log10(mstar_10)
#mstar_1 = np.log10(mstar_1)
#mstar_01 = np.log10(mstar_01)

fig=plt.figure(figsize=(15,10))
ax = plt.subplot(111)

#inverse relative error
weights = np.abs(np.asarray(r0_10)/np.asarray(r0_10_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(np.asarray(sfr_5_10)/np.asarray(sfr_1000_10), np.asarray(r0_10)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(np.asarray(sfr_5_10)/np.asarray(sfr_1000_10), weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(np.asarray(sfr_5_10)/np.asarray(sfr_1000_10), np.asarray(r0_10)*weights*np.asarray(r0_10)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(np.asarray(sfr_5_10)/np.asarray(sfr_1000_10), weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(np.asarray(sfr_5_10)/np.asarray(sfr_1000_10), np.asarray(r0_10), statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax.errorbar(bin_centers, hist, yerr=std/np.sqrt(n), c='darkmagenta', marker='s')#=50)

#inverse relative error n=1
weights = np.abs(np.asarray(r0_1)/np.asarray(r0_1_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(np.asarray(sfr_5_1)/np.asarray(sfr_1000_1), np.asarray(r0_1)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(np.asarray(sfr_5_1)/np.asarray(sfr_1000_1), weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(np.asarray(sfr_5_1)/np.asarray(sfr_1000_1), np.asarray(r0_1)*weights*np.asarray(r0_1)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(np.asarray(sfr_5_1)/np.asarray(sfr_1000_1), weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(np.asarray(sfr_5_1)/np.asarray(sfr_1000_1), np.asarray(r0_1), statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax.errorbar(bin_centers, hist, yerr=std/np.sqrt(n), c='k', marker='^')#=50)

#inverse relative error n=0.1
weights = np.abs(np.asarray(r0_01)/np.asarray(r0_01_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(np.asarray(sfr_5_01)/np.asarray(sfr_1000_01), np.asarray(r0_01)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(np.asarray(sfr_5_01)/np.asarray(sfr_1000_01), weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(np.asarray(sfr_5_01)/np.asarray(sfr_1000_01), np.asarray(r0_01)*weights*np.asarray(r0_01)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(np.asarray(sfr_5_01)/np.asarray(sfr_1000_01), weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(np.asarray(sfr_5_01)/np.asarray(sfr_1000_01), np.asarray(r0_01), statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax.errorbar(bin_centers, hist, yerr=std/np.sqrt(n), c='c', marker='o')#=50)
    
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='darkmagenta', marker='s', label=r'$n=10.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='k', marker='^', label=r'$n=1.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='c', marker='o', label=r'$n=0.1$')

#ax.set_xscale('log')
ax.set_yscale('log')
ax.set_ylim(4e2,7e4)
ax.legend(loc=0, fontsize=20)
ax.set_xlabel(r'SFR(5 Myr)/SFR(1 Gyr)')
ax.set_ylabel(r'correlation length $r_{0}$ [pc]')
plt.savefig('./plots/r0_sfr_ratio_2.pdf',bbox_inches='tight')


In [ ]:
#
from scipy.stats import binned_statistic as bs

bins = 5
range = None #(0,10)
#mstar_10 = np.log10(mstar_10)
#mstar_1 = np.log10(mstar_1)
#mstar_01 = np.log10(mstar_01)

fig=plt.figure(figsize=(15,10))
ax = plt.subplot(111)

#inverse relative error
weights = np.abs(np.asarray(r0_10)/np.asarray(r0_10_err))

log_sfr_10 = np.log10(np.asarray(sfr_5_10)/np.asarray(sfr_200_10))
tmp = np.asarray(sfr_5_10)/np.asarray(sfr_200_10) == 0.
log_sfr_10[tmp] = 1e-8

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(log_sfr_10, np.asarray(r0_10)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(log_sfr_10, weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(log_sfr_10, np.asarray(r0_10)*weights*np.asarray(r0_10)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(log_sfr_10, weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(log_sfr_10, np.asarray(r0_10), statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax.errorbar(10**bin_centers, hist, yerr=std/np.sqrt(n), c='darkmagenta', marker='s')#=50)

#inverse relative error n=1
weights = np.abs(np.asarray(r0_1)/np.asarray(r0_1_err))

log_sfr_1 = np.log10(np.asarray(sfr_5_1)/np.asarray(sfr_200_1))
tmp = np.asarray(sfr_5_1)/np.asarray(sfr_200_1) == 0.
log_sfr_1[tmp] = 1e-8

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(log_sfr_1, np.asarray(r0_1)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(log_sfr_1, weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(log_sfr_1, np.asarray(r0_1)*weights*np.asarray(r0_1)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(log_sfr_1, weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(log_sfr_1, np.asarray(r0_1), statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax.errorbar(10**bin_centers, hist, yerr=std/np.sqrt(n), c='k', marker='^')#=50)

#inverse relative error n=0.1
weights = np.abs(np.asarray(r0_01)/np.asarray(r0_01_err))

log_sfr_01 = np.log10(np.asarray(sfr_5_01)/np.asarray(sfr_200_01))
tmp = np.asarray(sfr_5_01)/np.asarray(sfr_200_01) == 0.
log_sfr_01[tmp] = 1e-8

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(log_sfr_01, np.asarray(r0_01)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(log_sfr_01, weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(log_sfr_01, np.asarray(r0_01)*weights*np.asarray(r0_01)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(log_sfr_01, weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(log_sfr_01, np.asarray(r0_01), statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax.errorbar(10**bin_centers, hist, yerr=std/np.sqrt(n), c='c', marker='o')#=50)
    
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='darkmagenta', marker='s', label=r'$n=10.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='k', marker='^', label=r'$n=1.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='c', marker='o', label=r'$n=0.1$')

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_ylim(4e2,7e4)
ax.legend(loc=0, fontsize=20)
ax.set_xlabel(r'SFR(5 Myr)/SFR(200 Myr)')
ax.set_ylabel(r'correlation length $r_{0}$ [pc]')
plt.savefig('./plots/r0_sfr_ratio_3.pdf',bbox_inches='tight')


In [ ]:
#
from scipy.stats import binned_statistic as bs

bins = 8
range = None #(0,10)
#mstar_10 = np.log10(mstar_10)
#mstar_1 = np.log10(mstar_1)
#mstar_01 = np.log10(mstar_01)


idx = np.where(np.asarray(gamma_1)>-.6)

fig=plt.figure(figsize=(15,10))
ax = plt.subplot(111)

#inverse relative error
weights = np.abs(np.asarray(gamma_10)/np.asarray(gamma_10_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(np.asarray(sfr_5_10)/np.asarray(sfr_10), np.asarray(gamma_10)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(np.asarray(sfr_5_10)/np.asarray(sfr_10), weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(np.asarray(sfr_5_10)/np.asarray(sfr_10), np.asarray(gamma_10)*weights*np.asarray(gamma_10)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(np.asarray(sfr_5_10)/np.asarray(sfr_10), weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(np.asarray(sfr_5_10)/np.asarray(sfr_10), np.asarray(gamma_10), statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax.errorbar(bin_centers, hist, yerr=std/np.sqrt(n), c='r', marker='s')#=50)

#inverse relative error n=1
weights = np.abs(np.asarray(gamma_1)[idx]/np.asarray(gamma_1_err)[idx])

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(np.asarray(sfr_5_1)[idx]/np.asarray(sfr_1)[idx], np.asarray(gamma_1)[idx], statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(np.asarray(sfr_5_1)[idx]/np.asarray(sfr_1)[idx], weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(np.asarray(sfr_5_1)[idx]/np.asarray(sfr_1)[idx], np.asarray(gamma_1)[idx]*weights*np.asarray(gamma_1)[idx]*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(np.asarray(sfr_5_1)[idx]/np.asarray(sfr_1)[idx], weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(np.asarray(sfr_5_1)[idx]/np.asarray(sfr_1)[idx], np.asarray(gamma_1)[idx], statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax.errorbar(bin_centers, hist, yerr=std/np.sqrt(n), c='k', marker='^')#=50)

#inverse relative error n=0.1
weights = np.abs(np.asarray(gamma_01)/np.asarray(gamma_01_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(np.asarray(sfr_5_01)/np.asarray(sfr_01), np.asarray(gamma_01)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(np.asarray(sfr_5_01)/np.asarray(sfr_01), weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(np.asarray(sfr_5_01)/np.asarray(sfr_01), np.asarray(gamma_01)*weights*np.asarray(gamma_01)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(np.asarray(sfr_5_01)/np.asarray(sfr_01), weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(np.asarray(sfr_5_01)/np.asarray(sfr_01), np.asarray(gamma_01), statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax.errorbar(bin_centers, hist, yerr=std/np.sqrt(n), c='b', marker='o')#=50)
    
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='r', marker='s', label=r'$n=10.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='k', marker='^', label=r'$n=1.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='b', marker='o', label=r'$n=0.1$')

#ax.set_xscale('log')
#ax.set_yscale('log')
ax.set_ylim(-0.025,0.225)
#ax.set_xlim(0.0,7)
ax.legend(loc=0, fontsize=20)
ax.set_xlabel(r'SFR(5 Myr)/SFR(100 Myr)')
ax.set_ylabel(r'powerlaw index $\gamma$')
plt.savefig('./plots/gamma_sfr_ratio.pdf',bbox_inches='tight')



In [ ]:
#
from scipy.stats import binned_statistic as bs

bins = 5
range = None #(0,10)
#mstar_10 = np.log10(mstar_10)
#mstar_1 = np.log10(mstar_1)
#mstar_01 = np.log10(mstar_01)


idx = np.where(np.asarray(gamma_1)>-.6)

fig=plt.figure(figsize=(15,10))
ax = plt.subplot(111)

#inverse relative error
weights = np.abs(np.asarray(gamma_10)/np.asarray(gamma_10_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(np.asarray(sfr_5_10)/np.asarray(sfr_1000_10), np.asarray(gamma_10)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(np.asarray(sfr_5_10)/np.asarray(sfr_1000_10), weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(np.asarray(sfr_5_10)/np.asarray(sfr_1000_10), np.asarray(gamma_10)*weights*np.asarray(gamma_10)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(np.asarray(sfr_5_10)/np.asarray(sfr_1000_10), weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(np.asarray(sfr_5_10)/np.asarray(sfr_1000_10), np.asarray(gamma_10), statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax.errorbar(bin_centers, hist, yerr=std/np.sqrt(n), c='darkmagenta', marker='s')#=50)

#inverse relative error n=1
weights = np.abs(np.asarray(gamma_1)[idx]/np.asarray(gamma_1_err)[idx])

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(np.asarray(sfr_5_1)[idx]/np.asarray(sfr_1000_1)[idx], np.asarray(gamma_1)[idx], statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(np.asarray(sfr_5_1)[idx]/np.asarray(sfr_1000_1)[idx], weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(np.asarray(sfr_5_1)[idx]/np.asarray(sfr_1000_1)[idx], np.asarray(gamma_1)[idx]*weights*np.asarray(gamma_1)[idx]*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(np.asarray(sfr_5_1)[idx]/np.asarray(sfr_1000_1)[idx], weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(np.asarray(sfr_5_1)[idx]/np.asarray(sfr_1000_1)[idx], np.asarray(gamma_1)[idx], statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax.errorbar(bin_centers, hist, yerr=std/np.sqrt(n), c='k', marker='^')#=50)

#inverse relative error n=0.1
weights = np.abs(np.asarray(gamma_01)/np.asarray(gamma_01_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(np.asarray(sfr_5_01)/np.asarray(sfr_1000_01), np.asarray(gamma_01)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(np.asarray(sfr_5_01)/np.asarray(sfr_1000_01), weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(np.asarray(sfr_5_01)/np.asarray(sfr_1000_01), np.asarray(gamma_01)*weights*np.asarray(gamma_01)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(np.asarray(sfr_5_01)/np.asarray(sfr_1000_01), weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(np.asarray(sfr_5_01)/np.asarray(sfr_1000_01), np.asarray(gamma_01), statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax.errorbar(bin_centers, hist, yerr=std/np.sqrt(n), c='c', marker='o')#=50)
    
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='darkmagenta', marker='s', label=r'$n=10.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='k', marker='^', label=r'$n=1.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='c', marker='o', label=r'$n=0.1$')

#ax.set_xscale('log')
#ax.set_yscale('log')
ax.set_ylim(-0.025,0.225)
#ax.set_xlim(0.0,7)
ax.legend(loc=0, fontsize=20)
ax.set_xlabel(r'SFR(5 Myr)/SFR(1 Gyr)')
ax.set_ylabel(r'powerlaw index $\gamma$')
plt.savefig('./plots/gamma_sfr_ratio_2.pdf',bbox_inches='tight')


In [ ]:
#
from scipy.stats import binned_statistic as bs

bins = 5
range = None #(0,10)
#mstar_10 = np.log10(mstar_10)
#mstar_1 = np.log10(mstar_1)
#mstar_01 = np.log10(mstar_01)

fig=plt.figure(figsize=(10,13.2))

gs = gridspec.GridSpec(2, 1)
gs.update(wspace=.0, hspace=0.)  # set the spacing between axes.

ax = plt.subplot(gs[0])

#inverse relative error
weights = np.abs(np.asarray(r0_10)/np.asarray(r0_10_err))

log_sfr_10 = np.log10(np.asarray(sfr_5_10)/np.asarray(sfr_200_10))
tmp = np.asarray(sfr_5_10)/np.asarray(sfr_200_10) == 0.
log_sfr_10[tmp] = 1e-8

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(log_sfr_10, np.asarray(r0_10)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(log_sfr_10, weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(log_sfr_10, np.asarray(r0_10)*weights*np.asarray(r0_10)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(log_sfr_10, weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(log_sfr_10, np.asarray(r0_10), statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax.errorbar(10**bin_centers, hist, yerr=std/np.sqrt(n), c='darkmagenta', marker='s')#=50)

#inverse relative error n=1
weights = np.abs(np.asarray(r0_1)/np.asarray(r0_1_err))

log_sfr_1 = np.log10(np.asarray(sfr_5_1)/np.asarray(sfr_200_1))
tmp = np.asarray(sfr_5_1)/np.asarray(sfr_200_1) == 0.
log_sfr_1[tmp] = 1e-8

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(log_sfr_1, np.asarray(r0_1)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(log_sfr_1, weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(log_sfr_1, np.asarray(r0_1)*weights*np.asarray(r0_1)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(log_sfr_1, weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(log_sfr_1, np.asarray(r0_1), statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax.errorbar(10**bin_centers, hist, yerr=std/np.sqrt(n), c='k', marker='^')#=50)

#inverse relative error n=0.1
weights = np.abs(np.asarray(r0_01)/np.asarray(r0_01_err))

log_sfr_01 = np.log10(np.asarray(sfr_5_01)/np.asarray(sfr_200_01))
tmp = np.asarray(sfr_5_01)/np.asarray(sfr_200_01) == 0.
log_sfr_01[tmp] = 1e-8

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(log_sfr_01, np.asarray(r0_01)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(log_sfr_01, weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(log_sfr_01, np.asarray(r0_01)*weights*np.asarray(r0_01)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(log_sfr_01, weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(log_sfr_01, np.asarray(r0_01), statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax.errorbar(10**bin_centers, hist, yerr=std/np.sqrt(n), c='c', marker='o')#=50)

ax.fill_between([0.07,2e1],y1=np.mean(r0_obs)-np.std(r0_obs)/np.sqrt(6),y2=np.mean(r0_obs)+np.std(r0_obs)/np.sqrt(6), color='lightgray', label='LEGUS (Grasha+2017)', zorder=-1,alpha=.5)
 
#ax.fill_between(,y1=0.12,y2=0.28, color='lightgray', label='LEGUS (Grasha+2017)', zorder=-1, alpha=.5)

ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='darkmagenta', marker='s', label=r'$n=10.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='k', marker='^', label=r'$n=1.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='c', marker='o', label=r'$n=0.1$')

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_ylim(5e2,3e4)
ax.set_xlim(0.07,20)
ax.legend(loc=0, fontsize=20)
#ax.set_xlabel(r'SFR(5 Myr)/SFR(200 Myr)')
ax.set_ylabel(r'correlation length $r_{0}$ [pc]')

#################################################################################

idx = np.where(np.asarray(gamma_1)>-.6)

ax = plt.subplot(gs[1])

#inverse relative error
weights = np.abs(np.asarray(gamma_10)/np.asarray(gamma_10_err))

# weight each fit by inverse of relative fit error
log_sfr_10 = np.log10(np.asarray(sfr_5_10)/np.asarray(sfr_200_10))
tmp = np.asarray(sfr_5_10)/np.asarray(sfr_200_10) == 0.
log_sfr_10[tmp] = 1e-8
weighted_sum, bin_edg, binnum = bs(log_sfr_10, np.asarray(gamma_10)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(log_sfr_10, weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(log_sfr_10, np.asarray(gamma_10)*weights*np.asarray(gamma_10)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(log_sfr_10, weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(log_sfr_10, np.asarray(gamma_10), statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax.errorbar(10**bin_centers, hist, yerr=std/np.sqrt(n), c='darkmagenta', marker='s')#=50)

#inverse relative error n=1
weights = np.abs(np.asarray(gamma_1)[idx]/np.asarray(gamma_1_err)[idx])

log_sfr_1 = np.log10(np.asarray(sfr_5_1)[idx]/np.asarray(sfr_200_1)[idx])
tmp = np.asarray(sfr_5_1)[idx]/np.asarray(sfr_200_1)[idx] == 0.
log_sfr_1[tmp] = 1e-8

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(log_sfr_1, np.asarray(gamma_1)[idx], statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(log_sfr_1, weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(log_sfr_1, np.asarray(gamma_1)[idx]*weights*np.asarray(gamma_1)[idx]*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(log_sfr_1, weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(log_sfr_1, np.asarray(gamma_1)[idx], statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax.errorbar(10**bin_centers, hist, yerr=std/np.sqrt(n), c='k', marker='^')#=50)

#inverse relative error n=0.1
weights = np.abs(np.asarray(gamma_01)/np.asarray(gamma_01_err))

log_sfr_01 = np.log10(np.asarray(sfr_5_01)/np.asarray(sfr_200_01))
tmp = np.asarray(sfr_5_01)/np.asarray(sfr_200_01) == 0.
log_sfr_01[tmp] = 1e-8

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(log_sfr_01, np.asarray(gamma_01)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(log_sfr_01, weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(log_sfr_01, np.asarray(gamma_01)*weights*np.asarray(gamma_01)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(log_sfr_01, weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(log_sfr_01, np.asarray(gamma_01), statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax.errorbar(10**bin_centers, hist, yerr=std/np.sqrt(n), c='c', marker='o')#=50)
    
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='darkmagenta', marker='s', label=r'$n=10.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='k', marker='^', label=r'$n=1.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='c', marker='o', label=r'$n=0.1$')

ax.fill_between([0.07,2e1],y1=0.12,y2=0.28, color='lightgray', label='LEGUS (Grasha+2017)', zorder=-1, alpha=.5)

ax.set_xscale('log')
#ax.set_yscale('log')
ax.set_ylim(-0.125,0.325)
ax.set_xlim(0.07,20)
#ax.legend(loc=0, fontsize=20)
ax.set_xlabel(r'SFR(5 Myr)/SFR(200 Myr)')
ax.set_ylabel(r'powerlaw index $\gamma$')
plt.savefig('./plots/gamma_r0_sfr_ratio.pdf',bbox_inches='tight')


In [ ]:
#
from scipy.stats import binned_statistic as bs

bins = 3
range = None #(0,10)
#mstar_10 = np.log10(mstar_10)
#mstar_1 = np.log10(mstar_1)
#mstar_01 = np.log10(mstar_01)

fig=plt.figure(figsize=(10,13.2))

gs = gridspec.GridSpec(2, 1)
gs.update(wspace=.0, hspace=0.)  # set the spacing between axes.

ax = plt.subplot(gs[0])

#inverse relative error
weights = np.abs(np.asarray(r0_10)/np.asarray(r0_10_err))

log_sfr_10 = np.log10(np.asarray(sfr_5_10)/np.asarray(sfr_200_10))
tmp = np.asarray(sfr_5_10)/np.asarray(sfr_200_10) == 0.
log_sfr_10[tmp] = 1e-8

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(log_sfr_10, np.asarray(r0_10)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(log_sfr_10, weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(log_sfr_10, np.asarray(r0_10)*weights*np.asarray(r0_10)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(log_sfr_10, weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(log_sfr_10, np.asarray(r0_10), statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax.errorbar(10**bin_centers, hist, yerr=std/np.sqrt(n), c='darkmagenta', marker='s')#=50)

#inverse relative error n=1
weights = np.abs(np.asarray(r0_1)/np.asarray(r0_1_err))

log_sfr_1 = np.log10(np.asarray(sfr_5_1)/np.asarray(sfr_200_1))
tmp = np.asarray(sfr_5_1)/np.asarray(sfr_200_1) == 0.
log_sfr_1[tmp] = 1e-8

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(log_sfr_1, np.asarray(r0_1)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(log_sfr_1, weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(log_sfr_1, np.asarray(r0_1)*weights*np.asarray(r0_1)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(log_sfr_1, weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(log_sfr_1, np.asarray(r0_1), statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax.errorbar(10**bin_centers, hist, yerr=std/np.sqrt(n), c='k', marker='^')#=50)

#inverse relative error n=0.1
weights = np.abs(np.asarray(r0_01)/np.asarray(r0_01_err))

log_sfr_01 = np.log10(np.asarray(sfr_5_01)/np.asarray(sfr_200_01))
tmp = np.asarray(sfr_5_01)/np.asarray(sfr_200_01) == 0.
log_sfr_01[tmp] = 1e-8

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(log_sfr_01, np.asarray(r0_01)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(log_sfr_01, weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(log_sfr_01, np.asarray(r0_01)*weights*np.asarray(r0_01)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(log_sfr_01, weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(log_sfr_01, np.asarray(r0_01), statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

index = np.where(np.isfinite(hist))
ax.errorbar(10**bin_centers[index], hist[index], yerr=std[index]/np.sqrt(n[index]), c='c', marker='o')#=50)

#ax.fill_between([0.07,2e1],y1=np.mean(r0_obs)-np.std(r0_obs)/np.sqrt(6),y2=np.mean(r0_obs)+np.std(r0_obs)/np.sqrt(6), color='lightgray', label='LEGUS (Grasha+2017)', zorder=-1,alpha=.5)
 
#ax.fill_between(,y1=0.12,y2=0.28, color='lightgray', label='LEGUS (Grasha+2017)', zorder=-1, alpha=.5)

ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='darkmagenta', marker='s', label=r'$n=10.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='k', marker='^', label=r'$n=1.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='c', marker='o', label=r'$n=0.1$')

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_ylim(5e2,3e4)
ax.set_xlim(0.07,20)
ax.legend(loc=0, fontsize=20)
#ax.set_xlabel(r'SFR(5 Myr)/SFR(200 Myr)')
ax.set_ylabel(r'correlation length $r_{0}$ [pc]')

#################################################################################

idx = np.where(np.asarray(gamma_1)>-.6)

ax = plt.subplot(gs[1])

#inverse relative error
weights = np.abs(np.asarray(gamma_10)/np.asarray(gamma_10_err))

# weight each fit by inverse of relative fit error
log_sfr_10 = np.log10(np.asarray(sfr_5_10)/np.asarray(sfr_200_10))
tmp = np.asarray(sfr_5_10)/np.asarray(sfr_200_10) == 0.
log_sfr_10[tmp] = 1e-8
weighted_sum, bin_edg, binnum = bs(log_sfr_10, np.asarray(gamma_10)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(log_sfr_10, weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(log_sfr_10, np.asarray(gamma_10)*weights*np.asarray(gamma_10)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(log_sfr_10, weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(log_sfr_10, np.asarray(gamma_10), statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax.errorbar(10**bin_centers, hist, yerr=std/np.sqrt(n), c='darkmagenta', marker='s')#=50)

#inverse relative error n=1
weights = np.abs(np.asarray(gamma_1)[idx]/np.asarray(gamma_1_err)[idx])

log_sfr_1 = np.log10(np.asarray(sfr_5_1)[idx]/np.asarray(sfr_200_1)[idx])
tmp = np.asarray(sfr_5_1)[idx]/np.asarray(sfr_200_1)[idx] == 0.
log_sfr_1[tmp] = 1e-8

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(log_sfr_1, np.asarray(gamma_1)[idx], statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(log_sfr_1, weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(log_sfr_1, np.asarray(gamma_1)[idx]*weights*np.asarray(gamma_1)[idx]*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(log_sfr_1, weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(log_sfr_1, np.asarray(gamma_1)[idx], statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax.errorbar(10**bin_centers, hist, yerr=std/np.sqrt(n), c='k', marker='^')#=50)

#inverse relative error n=0.1
weights = np.abs(np.asarray(gamma_01)/np.asarray(gamma_01_err))

log_sfr_01 = np.log10(np.asarray(sfr_5_01)/np.asarray(sfr_200_01))
tmp = np.asarray(sfr_5_01)/np.asarray(sfr_200_01) == 0.
log_sfr_01[tmp] = 1e-8

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(log_sfr_01, np.asarray(gamma_01)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(log_sfr_01, weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(log_sfr_01, np.asarray(gamma_01)*weights*np.asarray(gamma_01)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(log_sfr_01, weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(log_sfr_01, np.asarray(gamma_01), statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

index = np.where(np.isfinite(hist))
ax.errorbar(10**bin_centers[index], hist[index], yerr=std[index]/np.sqrt(n[index]), c='c', marker='o')#=50)
    
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='darkmagenta', marker='s', label=r'$n=10.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='k', marker='^', label=r'$n=1.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='c', marker='o', label=r'$n=0.1$')

#ax.fill_between([0.07,2e1],y1=0.12,y2=0.28, color='lightgray', label='LEGUS (Grasha+2017)', zorder=-1, alpha=.5)

ax.set_xscale('log')
#ax.set_yscale('log')
ax.set_ylim(-0.125,0.325)
ax.set_xlim(0.07,20)
#ax.legend(loc=0, fontsize=20)
ax.set_xlabel(r'SFR(5 Myr)/SFR(200 Myr)')
ax.set_ylabel(r'powerlaw index $\gamma$')
plt.savefig('./plots/gamma_r0_sfr_ratio_3.pdf',bbox_inches='tight')


In [ ]:
#
from scipy.stats import binned_statistic as bs

bins = 5
range = None #(0,10)
#mstar_10 = np.log10(mstar_10)
#mstar_1 = np.log10(mstar_1)
#mstar_01 = np.log10(mstar_01)

fig=plt.figure(figsize=(10,13.2))

gs = gridspec.GridSpec(2, 1)
gs.update(wspace=.0, hspace=0.)  # set the spacing between axes.

ax = plt.subplot(gs[0])

#inverse relative error
weights = np.abs(np.asarray(r0_10)/np.asarray(r0_10_err))

log_sfr_10 = np.log10(np.asarray(sfr_5_10)/np.asarray(sfr_200_10))
tmp = np.asarray(sfr_5_10)/np.asarray(sfr_200_10) == 0.
log_sfr_10[tmp] = 1e-8

ax.scatter(np.asarray(sfr_5_10)/np.asarray(sfr_200_10), np.asarray(r0_10), c='darkmagenta', marker='s')#=50)

#inverse relative error n=1

ax.scatter(np.asarray(sfr_5_1)/np.asarray(sfr_200_1), np.asarray(r0_1), c='k', marker='^')#=50)

#inverse relative error n=0.1

ax.scatter(np.asarray(sfr_5_01)/np.asarray(sfr_200_01), np.asarray(r0_01), c='c', marker='o')#=50)

#ax.fill_between([0.07,2e1],y1=np.mean(r0_obs)-np.std(r0_obs)/np.sqrt(6),y2=np.mean(r0_obs)+np.std(r0_obs)/np.sqrt(6), color='lightgray', label='LEGUS (Grasha+2017)', zorder=-1,alpha=.5)
 
#ax.fill_between(,y1=0.12,y2=0.28, color='lightgray', label='LEGUS (Grasha+2017)', zorder=-1, alpha=.5)

ax.scatter([], [], c='darkmagenta', marker='s', label=r'$n=10.0$')
ax.scatter([], [], c='k', marker='^', label=r'$n=1.0$')
ax.scatter([], [], c='c', marker='o', label=r'$n=0.1$')

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_ylim(5e2,3e4)
ax.set_xlim(0.07,20)
ax.legend(loc=0, fontsize=20)
#ax.set_xlabel(r'SFR(5 Myr)/SFR(200 Myr)')
ax.set_ylabel(r'correlation length $r_{0}$ [pc]')

#################################################################################

idx = np.where(np.asarray(gamma_1)>-.6)

ax = plt.subplot(gs[1])

ax.scatter(np.asarray(sfr_5_10)/np.asarray(sfr_200_10), np.asarray(gamma_10), c='darkmagenta', marker='s')#=50)

#inverse relative error n=1

ax.scatter(np.asarray(sfr_5_1)[idx]/np.asarray(sfr_200_1)[idx], np.asarray(gamma_1)[idx], c='k', marker='^')#=50)

#inverse relative error n=0.1

ax.scatter(np.asarray(sfr_5_01)/np.asarray(sfr_200_01), np.asarray(gamma_01), c='c', marker='o')#=50)
    
ax.scatter([np.nan], [np.nan], c='darkmagenta', marker='s', label=r'$n=10.0$')
ax.scatter([np.nan], [np.nan], c='k', marker='^', label=r'$n=1.0$')
ax.scatter([np.nan], [np.nan], c='c', marker='o', label=r'$n=0.1$')

#ax.fill_between([0.07,2e1],y1=0.12,y2=0.28, color='lightgray', label='LEGUS (Grasha+2017)', zorder=-1, alpha=.5)

ax.set_xscale('log')
#ax.set_yscale('log')
ax.set_ylim(-0.125,0.325)
ax.set_xlim(0.07,20)
#ax.legend(loc=0, fontsize=20)
ax.set_xlabel(r'SFR(5 Myr)/SFR(200 Myr)')
ax.set_ylabel(r'powerlaw index $\gamma$')
plt.savefig('./plots/gamma_r0_sfr_ratio_unbinned.pdf',bbox_inches='tight')



In [ ]:
# paper plot
from scipy.stats import binned_statistic as bs


fig=plt.figure(figsize=(18,12))
gs = gridspec.GridSpec(2, 2)
gs.update(wspace=.0, hspace=0.)  # set the spacing between axes.

ax1 = plt.subplot(gs[0])
ax1.set_ylabel(r'correlation length $r_{0}$ [pc]')
ax1.set_yscale('log')
ax1.set_xscale('log')
ax1.set_xticklabels('')

ax2 = plt.subplot(gs[1])
ax2.set_yscale('log')
ax2.set_yticklabels('')
ax2.set_xticklabels('')

ax3 = plt.subplot(gs[2])
ax3.set_ylabel(r'powerlaw index $\gamma$')
ax3.set_xlabel(r'$M_{\star}$ [$M_{\odot}$]')
ax3.set_xscale('log')

ax4 = plt.subplot(gs[3])
ax4.set_yticklabels('')
ax4.set_xlabel(r'SFR [$M_{\odot}/$yr]')


##### correlation length vs. mstar #####

bins = 5
range = None #(0,10)

#inverse relative error
weights = np.abs(np.asarray(r0_10)/np.asarray(r0_10_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(np.log10(mstar_10), np.asarray(r0_10)*weights, statistic=np.sum, bins=bins)
weight_hist, _, _ = bs(np.log10(mstar_10), weights, statistic=np.sum, bins=bins)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(np.log10(mstar_10), np.asarray(r0_10)*weights*np.asarray(r0_10)*weights, statistic=np.sum, bins=bins)
weight_hist_sq, _, _ = bs(np.log10(mstar_10), weights*weights, statistic=np.sum, bins=bins)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(np.log10(mstar_10), np.asarray(r0_10), statistic='count', bins=bins)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax1.errorbar(10**bin_centers, hist, yerr=std/np.sqrt(n), c='darkmagenta', marker='s')#=50)

#inverse relative error n=1
weights = np.abs(np.asarray(r0_1)/np.asarray(r0_1_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(np.log10(mstar_1), np.asarray(r0_1)*weights, statistic=np.sum, bins=bins)
weight_hist, _, _ = bs(np.log10(mstar_1), weights, statistic=np.sum, bins=bins)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(np.log10(mstar_1), np.asarray(r0_1)*weights*np.asarray(r0_1)*weights, statistic=np.sum, bins=bins)
weight_hist_sq, _, _ = bs(np.log10(mstar_1), weights*weights, statistic=np.sum, bins=bins)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(np.log10(mstar_1), np.asarray(r0_1), statistic='count', bins=bins)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax1.errorbar(10**bin_centers, hist, yerr=std/np.sqrt(n), c='k', marker='^')#=50)

#inverse relative error n=0.1
weights = np.abs(np.asarray(r0_01)/np.asarray(r0_01_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(np.log10(mstar_01), np.asarray(r0_01)*weights, statistic=np.sum, bins=bins)
weight_hist, _, _ = bs(np.log10(mstar_01), weights, statistic=np.sum, bins=bins)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(np.log10(mstar_01), np.asarray(r0_01)*weights*np.asarray(r0_01)*weights, statistic=np.sum, bins=bins)
weight_hist_sq, _, _ = bs(np.log10(mstar_01), weights*weights, statistic=np.sum, bins=bins)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(np.log10(mstar_01), np.asarray(r0_01), statistic='count', bins=bins)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax1.errorbar(10**bin_centers, hist, yerr=std/np.sqrt(n), c='c', marker='o')#=50)

ax1.errorbar(mstar_obs,r0_obs,yerr=[r0_obs_err_l,r0_obs_err_h],c='lightgray',fmt='*',markersize=30,label='LEGUS (Grasha+2017)')

ax1.set_xlim(1e7,7.0e10)
ax1.set_ylim(1e2,3e4)
ax1.set_yticks([1e3,1e4])

##### correlation length vs. SFR #####

bins = 6
#inverse relative error
weights = np.abs(np.asarray(r0_10)/np.asarray(r0_10_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(sfr_10, np.asarray(r0_10)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(sfr_10, weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(sfr_10, np.asarray(r0_10)*weights*np.asarray(r0_10)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(sfr_10, weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(sfr_10, np.asarray(r0_10), statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax2.errorbar(bin_centers, hist, yerr=std/np.sqrt(n), c='darkmagenta', marker='s')#=50)

#inverse relative error n=1
weights = np.abs(np.asarray(r0_1)/np.asarray(r0_1_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(sfr_1, np.asarray(r0_1)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(sfr_1, weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(sfr_1, np.asarray(r0_1)*weights*np.asarray(r0_1)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(sfr_1, weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(sfr_1, np.asarray(r0_1), statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax2.errorbar(bin_centers, hist, yerr=std/np.sqrt(n), c='k', marker='^')#=50)

#inverse relative error n=0.1
weights = np.abs(np.asarray(r0_01)/np.asarray(r0_01_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(sfr_01, np.asarray(r0_01)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(sfr_01, weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(sfr_01, np.asarray(r0_01)*weights*np.asarray(r0_01)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(sfr_01, weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(sfr_01, np.asarray(r0_01), statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax2.errorbar(bin_centers, hist, yerr=std/np.sqrt(n), c='c', marker='o')#=50)

ax2.errorbar(sfr_obs,r0_obs,yerr=[r0_obs_err_l,r0_obs_err_h],c='lightgray',fmt='*',markersize=30, label='LEGUS (Grasha+2017)')

ax2.set_ylim(1e2,3e4)
ax2.set_xlim(0.0,7)

##### gamma vs. mstar #####

bins = 5

#inverse relative error
weights = np.abs(np.asarray(gamma_10)/np.asarray(gamma_10_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(np.log10(mstar_10), np.asarray(gamma_10)*weights, statistic=np.sum, bins=bins)
weight_hist, _, _ = bs(np.log10(mstar_10), weights, statistic=np.sum, bins=bins)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(np.log10(mstar_10), np.asarray(gamma_10)*weights*np.asarray(gamma_10)*weights, statistic=np.sum, bins=bins)
weight_hist_sq, _, _ = bs(np.log10(mstar_10), weights*weights, statistic=np.sum, bins=bins)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(np.log10(mstar_10), np.asarray(gamma_10), statistic='count', bins=bins)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax3.errorbar(10**bin_centers, hist, yerr=std/np.sqrt(n), c='darkmagenta', marker='s')#=50)

#inverse relative error n=1
weights = np.abs(np.asarray(gamma_1)/np.asarray(gamma_1_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(np.log10(mstar_1), np.asarray(gamma_1)*weights, statistic=np.sum, bins=bins)
weight_hist, _, _ = bs(np.log10(mstar_1), weights, statistic=np.sum, bins=bins)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(np.log10(mstar_1), np.asarray(gamma_1)*weights*np.asarray(gamma_1)*weights, statistic=np.sum, bins=bins)
weight_hist_sq, _, _ = bs(np.log10(mstar_1), weights*weights, statistic=np.sum, bins=bins)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(np.log10(mstar_1), np.asarray(gamma_1), statistic='count', bins=bins)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax3.errorbar(10**bin_centers, hist, yerr=std/np.sqrt(n), c='k', marker='^')#=50)

#inverse relative error n=0.1
weights = np.abs(np.asarray(gamma_01)/np.asarray(gamma_01_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(np.log10(mstar_01), np.asarray(gamma_01)*weights, statistic=np.sum, bins=bins)
weight_hist, _, _ = bs(np.log10(mstar_01), weights, statistic=np.sum, bins=bins)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(np.log10(mstar_01), np.asarray(gamma_01)*weights*np.asarray(gamma_01)*weights, statistic=np.sum, bins=bins)
weight_hist_sq, _, _ = bs(np.log10(mstar_01), weights*weights, statistic=np.sum, bins=bins)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(np.log10(mstar_01), np.asarray(gamma_01), statistic='count', bins=bins)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax3.errorbar(10**bin_centers, hist, yerr=std/np.sqrt(n), c='c', marker='o')#=50)

ax3.fill_between([1e7,7e10],y1=0.12,y2=0.28, color='lightgray', label='LEGUS (Grasha+2017)', zorder=-1, alpha=.5)

ax3.set_xlim(1e7,7e10)
ax3.set_ylim(-0.025,0.3)
#ax3.set_yticks([0,0.1,0.2])

##### gamma vs. SFR ####

bins = 6
range = None #(0,10)
#mstar_10 = np.log10(mstar_10)
#mstar_1 = np.log10(mstar_1)
#mstar_01 = np.log10(mstar_01)

idx = np.where(np.asarray(gamma_1)>-.6)

#inverse relative error
weights = np.abs(np.asarray(gamma_10)/np.asarray(gamma_10_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(sfr_10, np.asarray(gamma_10)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(sfr_10, weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(sfr_10, np.asarray(gamma_10)*weights*np.asarray(gamma_10)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(sfr_10, weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(sfr_10, np.asarray(gamma_10), statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax4.errorbar(bin_centers, hist, yerr=std/np.sqrt(n), c='darkmagenta', marker='s')#=50)

#inverse relative error n=1
weights = np.abs(np.asarray(gamma_1)[idx]/np.asarray(gamma_1_err)[idx])

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(np.asarray(sfr_1)[idx], np.asarray(gamma_1)[idx]*weights, statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(np.asarray(sfr_1)[idx], weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(np.asarray(sfr_1)[idx], np.asarray(gamma_1)[idx]*weights*np.asarray(gamma_1)[idx]*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(np.asarray(sfr_1)[idx], weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(np.asarray(sfr_1)[idx], np.asarray(gamma_1)[idx], statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax4.errorbar(bin_centers, hist, yerr=std/np.sqrt(n), c='k', marker='^')#=50)

#inverse relative error n=0.1
weights = np.abs(np.asarray(gamma_01)/np.asarray(gamma_01_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(sfr_01, np.asarray(gamma_01)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(sfr_01, weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(sfr_01, np.asarray(gamma_01)*weights*np.asarray(gamma_01)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(sfr_01, weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(sfr_01, np.asarray(gamma_01), statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax4.errorbar(bin_centers, hist, yerr=std/np.sqrt(n), c='c', marker='o')#=50)

   
ax4.errorbar([np.nan], [np.nan], yerr=[np.nan], c='darkmagenta', marker='s', label=r'$n=10.0$')
ax4.errorbar([np.nan], [np.nan], yerr=[np.nan], c='k', marker='^', label=r'$n=1.0$')
ax4.errorbar([np.nan], [np.nan], yerr=[np.nan], c='c', marker='o', label=r'$n=0.1$')

ax4.fill_between([0,7],y1=0.12,y2=0.28, color='lightgray', label='LEGUS (Grasha+2017)', zorder=-1, alpha=.5)

ax4.set_ylim(-0.025,0.3)
ax4.set_xlim(0.0,7)

ax4.legend(loc=0, fontsize=20)
plt.savefig('two_point_4_panel_new.pdf', bbox_inches='tight')

In [ ]:
# paper plot
from scipy.stats import binned_statistic as bs
from scipy import stats

fig=plt.figure(figsize=(18,12))
gs = gridspec.GridSpec(2, 2)
gs.update(wspace=.0, hspace=0.)  # set the spacing between axes.

ax1 = plt.subplot(gs[0])
ax1.set_ylabel(r'correlation length $r_{0}$ [pc]')
ax1.set_yscale('log')
ax1.set_xscale('log')
ax1.set_xticklabels('')

ax2 = plt.subplot(gs[1])
ax2.set_yscale('log')
ax2.set_yticklabels('')
ax2.set_xticklabels('')

ax3 = plt.subplot(gs[2])
ax3.set_ylabel(r'powerlaw index $\gamma$')
ax3.set_xlabel(r'$M_{\star}$ [$M_{\odot}$]')
ax3.set_xscale('log')

ax4 = plt.subplot(gs[3])
ax4.set_yticklabels('')
ax4.set_xlabel(r'SFR [$M_{\odot}/$yr]')


##### correlation length vs. mstar #####

bins = 5
range = None #(0,10)


ax1.errorbar(10**bin_centers, hist, yerr=std/np.sqrt(n), c='darkmagenta', marker='s')#=50)

log_sfr_10 = np.log10(np.asarray(sfr_5_10)/np.asarray(sfr_200_10))
tmp = np.asarray(sfr_5_10)/np.asarray(sfr_200_10) == 0.
log_sfr_10[tmp] = 1e-8
xd = log_sfr_10
yd = np.log10(np.asarray(r0_10))
slope_10, intercept_10, r_value_10, p_value_10, std_err = stats.linregress(xd,yd)

plt.plot(x,10**(slope_10*np.log10(x)+intercept_10), c='darkmagenta',lw=6,label=r'p-value=%.2f'%p_value_10)


#inverse relative error n=1
weights = np.abs(np.asarray(r0_1)/np.asarray(r0_1_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(np.log10(mstar_1), np.asarray(r0_1)*weights, statistic=np.sum, bins=bins)
weight_hist, _, _ = bs(np.log10(mstar_1), weights, statistic=np.sum, bins=bins)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(np.log10(mstar_1), np.asarray(r0_1)*weights*np.asarray(r0_1)*weights, statistic=np.sum, bins=bins)
weight_hist_sq, _, _ = bs(np.log10(mstar_1), weights*weights, statistic=np.sum, bins=bins)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(np.log10(mstar_1), np.asarray(r0_1), statistic='count', bins=bins)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax1.errorbar(10**bin_centers, hist, yerr=std/np.sqrt(n), c='k', marker='^')#=50)

#inverse relative error n=0.1
weights = np.abs(np.asarray(r0_01)/np.asarray(r0_01_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(np.log10(mstar_01), np.asarray(r0_01)*weights, statistic=np.sum, bins=bins)
weight_hist, _, _ = bs(np.log10(mstar_01), weights, statistic=np.sum, bins=bins)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(np.log10(mstar_01), np.asarray(r0_01)*weights*np.asarray(r0_01)*weights, statistic=np.sum, bins=bins)
weight_hist_sq, _, _ = bs(np.log10(mstar_01), weights*weights, statistic=np.sum, bins=bins)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(np.log10(mstar_01), np.asarray(r0_01), statistic='count', bins=bins)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax1.errorbar(10**bin_centers, hist, yerr=std/np.sqrt(n), c='c', marker='o')#=50)

ax1.errorbar(mstar_obs,r0_obs,yerr=[r0_obs_err_l,r0_obs_err_h],c='lightgray',fmt='*',markersize=30,label='LEGUS (Grasha+2017)')

ax1.set_xlim(1e7,7.0e10)
ax1.set_ylim(1e2,3e4)
ax1.set_yticks([1e3,1e4])

##### correlation length vs. SFR #####

bins = 6
#inverse relative error
weights = np.abs(np.asarray(r0_10)/np.asarray(r0_10_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(sfr_10, np.asarray(r0_10)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(sfr_10, weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(sfr_10, np.asarray(r0_10)*weights*np.asarray(r0_10)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(sfr_10, weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(sfr_10, np.asarray(r0_10), statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax2.errorbar(bin_centers, hist, yerr=std/np.sqrt(n), c='darkmagenta', marker='s')#=50)

#inverse relative error n=1
weights = np.abs(np.asarray(r0_1)/np.asarray(r0_1_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(sfr_1, np.asarray(r0_1)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(sfr_1, weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(sfr_1, np.asarray(r0_1)*weights*np.asarray(r0_1)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(sfr_1, weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(sfr_1, np.asarray(r0_1), statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax2.errorbar(bin_centers, hist, yerr=std/np.sqrt(n), c='k', marker='^')#=50)

#inverse relative error n=0.1
weights = np.abs(np.asarray(r0_01)/np.asarray(r0_01_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(sfr_01, np.asarray(r0_01)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(sfr_01, weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(sfr_01, np.asarray(r0_01)*weights*np.asarray(r0_01)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(sfr_01, weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(sfr_01, np.asarray(r0_01), statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax2.errorbar(bin_centers, hist, yerr=std/np.sqrt(n), c='c', marker='o')#=50)

ax2.errorbar(sfr_obs,r0_obs,yerr=[r0_obs_err_l,r0_obs_err_h],c='lightgray',fmt='*',markersize=30, label='LEGUS (Grasha+2017)')

ax2.set_ylim(1e2,3e4)
ax2.set_xlim(0.0,7)

##### gamma vs. mstar #####

bins = 5

#inverse relative error
weights = np.abs(np.asarray(gamma_10)/np.asarray(gamma_10_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(np.log10(mstar_10), np.asarray(gamma_10)*weights, statistic=np.sum, bins=bins)
weight_hist, _, _ = bs(np.log10(mstar_10), weights, statistic=np.sum, bins=bins)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(np.log10(mstar_10), np.asarray(gamma_10)*weights*np.asarray(gamma_10)*weights, statistic=np.sum, bins=bins)
weight_hist_sq, _, _ = bs(np.log10(mstar_10), weights*weights, statistic=np.sum, bins=bins)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(np.log10(mstar_10), np.asarray(gamma_10), statistic='count', bins=bins)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax3.errorbar(10**bin_centers, hist, yerr=std/np.sqrt(n), c='darkmagenta', marker='s')#=50)

#inverse relative error n=1
weights = np.abs(np.asarray(gamma_1)/np.asarray(gamma_1_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(np.log10(mstar_1), np.asarray(gamma_1)*weights, statistic=np.sum, bins=bins)
weight_hist, _, _ = bs(np.log10(mstar_1), weights, statistic=np.sum, bins=bins)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(np.log10(mstar_1), np.asarray(gamma_1)*weights*np.asarray(gamma_1)*weights, statistic=np.sum, bins=bins)
weight_hist_sq, _, _ = bs(np.log10(mstar_1), weights*weights, statistic=np.sum, bins=bins)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(np.log10(mstar_1), np.asarray(gamma_1), statistic='count', bins=bins)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax3.errorbar(10**bin_centers, hist, yerr=std/np.sqrt(n), c='k', marker='^')#=50)

#inverse relative error n=0.1
weights = np.abs(np.asarray(gamma_01)/np.asarray(gamma_01_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(np.log10(mstar_01), np.asarray(gamma_01)*weights, statistic=np.sum, bins=bins)
weight_hist, _, _ = bs(np.log10(mstar_01), weights, statistic=np.sum, bins=bins)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(np.log10(mstar_01), np.asarray(gamma_01)*weights*np.asarray(gamma_01)*weights, statistic=np.sum, bins=bins)
weight_hist_sq, _, _ = bs(np.log10(mstar_01), weights*weights, statistic=np.sum, bins=bins)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(np.log10(mstar_01), np.asarray(gamma_01), statistic='count', bins=bins)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax3.errorbar(10**bin_centers, hist, yerr=std/np.sqrt(n), c='c', marker='o')#=50)

ax3.fill_between([1e7,7e10],y1=0.12,y2=0.28, color='lightgray', label='LEGUS (Grasha+2017)', zorder=-1, alpha=.5)

ax3.set_xlim(1e7,7e10)
ax3.set_ylim(-0.025,0.3)
#ax3.set_yticks([0,0.1,0.2])

##### gamma vs. SFR ####

bins = 6
range = None #(0,10)
#mstar_10 = np.log10(mstar_10)
#mstar_1 = np.log10(mstar_1)
#mstar_01 = np.log10(mstar_01)

idx = np.where(np.asarray(gamma_1)>-.6)

#inverse relative error
weights = np.abs(np.asarray(gamma_10)/np.asarray(gamma_10_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(sfr_10, np.asarray(gamma_10)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(sfr_10, weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(sfr_10, np.asarray(gamma_10)*weights*np.asarray(gamma_10)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(sfr_10, weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(sfr_10, np.asarray(gamma_10), statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax4.errorbar(bin_centers, hist, yerr=std/np.sqrt(n), c='darkmagenta', marker='s')#=50)

#inverse relative error n=1
weights = np.abs(np.asarray(gamma_1)[idx]/np.asarray(gamma_1_err)[idx])

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(np.asarray(sfr_1)[idx], np.asarray(gamma_1)[idx]*weights, statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(np.asarray(sfr_1)[idx], weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(np.asarray(sfr_1)[idx], np.asarray(gamma_1)[idx]*weights*np.asarray(gamma_1)[idx]*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(np.asarray(sfr_1)[idx], weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(np.asarray(sfr_1)[idx], np.asarray(gamma_1)[idx], statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax4.errorbar(bin_centers, hist, yerr=std/np.sqrt(n), c='k', marker='^')#=50)

#inverse relative error n=0.1
weights = np.abs(np.asarray(gamma_01)/np.asarray(gamma_01_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(sfr_01, np.asarray(gamma_01)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist, _, _ = bs(sfr_01, weights, statistic=np.sum, bins=bins, range=range)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(sfr_01, np.asarray(gamma_01)*weights*np.asarray(gamma_01)*weights, statistic=np.sum, bins=bins, range=range)
weight_hist_sq, _, _ = bs(sfr_01, weights*weights, statistic=np.sum, bins=bins, range=range)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(sfr_01, np.asarray(gamma_01), statistic='count', bins=bins, range=range)

std = np.sqrt(hist_sq - hist)

bin_width = (bin_edg[1] - bin_edg[0])
bin_centers = bin_edg[1:] - bin_width/2

ax4.errorbar(bin_centers, hist, yerr=std/np.sqrt(n), c='c', marker='o')#=50)

   
ax4.errorbar([np.nan], [np.nan], yerr=[np.nan], c='darkmagenta', marker='s', label=r'$n=10.0$')
ax4.errorbar([np.nan], [np.nan], yerr=[np.nan], c='k', marker='^', label=r'$n=1.0$')
ax4.errorbar([np.nan], [np.nan], yerr=[np.nan], c='c', marker='o', label=r'$n=0.1$')

ax4.fill_between([0,7],y1=0.12,y2=0.28, color='lightgray', label='LEGUS (Grasha+2017)', zorder=-1, alpha=.5)

ax4.set_ylim(-0.025,0.3)
ax4.set_xlim(0.0,7)

ax4.legend(loc=0, fontsize=20)
plt.savefig('two_point_mstar_SFR.pdf', bbox_inches='tight')

In [ ]:
#
from scipy.stats import binned_statistic as bs

bins = 1
#mstar_10 = np.log10(mstar_10)
#mstar_1 = np.log10(mstar_1)
#mstar_01 = np.log10(mstar_01)

fig=plt.figure(figsize=(15,10))
ax = plt.subplot(111)

# average scale length in mstar bins
#inverse relative error
weights = np.abs(np.asarray(r0_10)/np.asarray(r0_10_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(np.log10(mstar_10), np.asarray(r0_10)*weights, statistic=np.sum, bins=bins)
weight_hist, _, _ = bs(np.log10(mstar_10), weights, statistic=np.sum, bins=bins)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(np.log10(mstar_10), np.asarray(r0_10)*weights*np.asarray(r0_10)*weights, statistic=np.sum, bins=bins)
weight_hist_sq, _, _ = bs(np.log10(mstar_10), weights*weights, statistic=np.sum, bins=bins)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(np.log10(mstar_10), np.asarray(r0_10), statistic='count', bins=bins)

r0_std = np.sqrt(hist_sq - hist) / np.sqrt(n)
r0_hist = hist

#average gamma in mstar bins
#inverse relative error
weights = np.abs(np.asarray(gamma_10)/np.asarray(gamma_10_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(np.log10(mstar_10), np.asarray(gamma_10)*weights, statistic=np.sum, bins=bin_edg)
weight_hist, _, _ = bs(np.log10(mstar_10), weights, statistic=np.sum, bins=bin_edg)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(np.log10(mstar_10), np.asarray(gamma_10)*weights*np.asarray(gamma_10)*weights, statistic=np.sum, bins=bin_edg)
weight_hist_sq, _, _ = bs(np.log10(mstar_10), weights*weights, statistic=np.sum, bins=bin_edg)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(np.log10(mstar_10), np.asarray(gamma_10), statistic='count', bins=bin_edg)

gamma_std = np.sqrt(hist_sq - hist) / np.sqrt(n)
gamma_hist = hist

ax.errorbar(gamma_hist, r0_hist, xerr=gamma_std, yerr=r0_std, c='darkmagenta', marker='s', fmt='s', markersize=17.5)

# average scale length in mstar bins
#inverse relative error
weights = np.abs(np.asarray(r0_1)/np.asarray(r0_1_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(np.log10(mstar_1), np.asarray(r0_1)*weights, statistic=np.sum, bins=bins)
weight_hist, _, _ = bs(np.log10(mstar_1), weights, statistic=np.sum, bins=bins)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(np.log10(mstar_1), np.asarray(r0_1)*weights*np.asarray(r0_1)*weights, statistic=np.sum, bins=bins)
weight_hist_sq, _, _ = bs(np.log10(mstar_1), weights*weights, statistic=np.sum, bins=bins)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(np.log10(mstar_1), np.asarray(r0_1), statistic='count', bins=bins)

r0_std = np.sqrt(hist_sq - hist) / np.sqrt(n)
r0_hist = hist

#average gamma in mstar bins
#inverse relative error
weights = np.abs(np.asarray(gamma_1)/np.asarray(gamma_1_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(np.log10(mstar_1), np.asarray(gamma_1)*weights, statistic=np.sum, bins=bin_edg)
weight_hist, _, _ = bs(np.log10(mstar_1), weights, statistic=np.sum, bins=bin_edg)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(np.log10(mstar_1), np.asarray(gamma_1)*weights*np.asarray(gamma_1)*weights, statistic=np.sum, bins=bin_edg)
weight_hist_sq, _, _ = bs(np.log10(mstar_1), weights*weights, statistic=np.sum, bins=bin_edg)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(np.log10(mstar_1), np.asarray(gamma_1), statistic='count', bins=bin_edg)

gamma_std = np.sqrt(hist_sq - hist) / np.sqrt(n)
gamma_hist = hist

ax.errorbar(gamma_hist, r0_hist, xerr=gamma_std, yerr=r0_std, c='k', marker='^', fmt='^', markersize=17.5)


# average scale length in mstar bins
#inverse relative error
weights = np.abs(np.asarray(r0_01)/np.asarray(r0_01_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(np.log10(mstar_01), np.asarray(r0_01)*weights, statistic=np.sum, bins=bins)
weight_hist, _, _ = bs(np.log10(mstar_01), weights, statistic=np.sum, bins=bins)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(np.log10(mstar_01), np.asarray(r0_01)*weights*np.asarray(r0_01)*weights, statistic=np.sum, bins=bins)
weight_hist_sq, _, _ = bs(np.log10(mstar_01), weights*weights, statistic=np.sum, bins=bins)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(np.log10(mstar_01), np.asarray(r0_01), statistic='count', bins=bins)

r0_std = np.sqrt(hist_sq - hist) / np.sqrt(n)
r0_hist = hist

#average gamma in mstar bins
#inverse relative error
weights = np.abs(np.asarray(gamma_01)/np.asarray(gamma_01_err))

# weight each fit by inverse of relative fit error
weighted_sum, bin_edg, binnum = bs(np.log10(mstar_01), np.asarray(gamma_01)*weights, statistic=np.sum, bins=bin_edg)
weight_hist, _, _ = bs(np.log10(mstar_01), weights, statistic=np.sum, bins=bin_edg)

hist = weighted_sum / weight_hist

weighted_sum_sq, _, _ = bs(np.log10(mstar_01), np.asarray(gamma_01)*weights*np.asarray(gamma_01)*weights, statistic=np.sum, bins=bin_edg)
weight_hist_sq, _, _ = bs(np.log10(mstar_01), weights*weights, statistic=np.sum, bins=bin_edg)

hist_sq = weighted_sum_sq / weight_hist_sq

n, _, _ = bs(np.log10(mstar_01), np.asarray(gamma_01), statistic='count', bins=bin_edg)

gamma_std = np.sqrt(hist_sq - hist) / np.sqrt(n)
gamma_hist = hist

ax.errorbar(gamma_hist, r0_hist, xerr=gamma_std, yerr=r0_std, c='c', marker='o', fmt='o', markersize=17.5)

ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='darkmagenta', marker='s', label=r'$n=10.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='k', marker='^', label=r'$n=1.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='c', marker='o', label=r'$n=0.1$')


#ax.fill_between([-0.1,0.5],y1=np.mean(r0_obs)-np.std(r0_obs)/np.sqrt(6),y2=np.mean(r0_obs)+np.std(r0_obs)/np.sqrt(6), color='lightgray', label='LEGUS (Grasha+2017)',zorder=-1, alpha=.5)
#ax.fill_between([0.12,0.28],y1=3,y2=3e4, color='lightgray', zorder=-1,alpha=.5)
ax.fill_between([0.12,0.28],y1=np.mean(r0_obs)-np.std(r0_obs)/np.sqrt(6),y2=np.mean(r0_obs)+np.std(r0_obs)/np.sqrt(6), color='lightgray', label='LEGUS (Grasha+2017)',zorder=-1,alpha=.5)



ax.set_xlim(0.025,0.3)
ax.set_ylim(7e2,2e4)
#ax.set_xscale('log')
ax.set_yscale('log')
ax.legend(loc=0, fontsize=20)
ax.set_xlabel(r'powerlaw index $\gamma$')
ax.set_ylabel(r'correlation length $r_{0}$ [pc]')
plt.savefig('./plots/gamma_vs_r0_binned.pdf',bbox_inches='tight')


In [ ]:
np.median(np.asarray(sfr_5_10)/np.asarray(sfr_10))

# Only redshift zero snaps

In [ ]:
sims_good = ['g3.49e11','g3.55e11','g3.71e11','g7.08e11','g7.55e11','g7.66e11','g8.26e11'] #,'g2.19e11','g3.21e11'

fig=plt.figure(figsize=(15,10))
ax = plt.subplot(111)
    
for sim in sims_good:
    thresholds = glob.glob(sim+'_*.01024_two_point.dat')
    for thresh in thresholds:
        
        n_temp = thresh.split('_')[1]
        e_temp = thresh.split('_')[2]
        n = n_temp[1:]
        e = e_temp[1:5]
        
        data = pickle.load(open(thresh,'rb'), encoding='latin1') 
        mstar = data['mstar']
        
        data = pickle.load(open('./plots/'+thresh+'_0.05Gyr_plot.dat','rb'))
        popt = data['popt']
        pcov = data['pcov']
        
        gamma = popt[0]
        r0 = popt[1]
        a = popt[2]

        gamma_err = np.sqrt( pcov[0][0] )
        r0_err = np.sqrt( pcov[1][1] )
        a_err = np.sqrt( pcov[2][2] )
        
        if n == '10.0':
            ax.errorbar(mstar, gamma, yerr=gamma_err, c='r', marker='s')#=50)
        elif n == '1.0':
            ax.errorbar(mstar, gamma, yerr=gamma_err, c='k', marker='^')
        elif n == '0.1':
            if e == '0.04':
                ax.errorbar(mstar, gamma, yerr=gamma_err, c='b', marker='o')
        
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='r', marker='s', label=r'$n=10.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='k', marker='^', label=r'$n=1.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='b', marker='o', label=r'$n=0.1$')

ax.fill_between([3e9,7e10],y1=0.12,y2=0.28, color='lightgray', label='LEGUS (Grasha+2017)')

ax.set_xlim(3e9,7e10)
ax.set_xscale('log')
#ax.set_yscale('log')
ax.legend(loc=0, fontsize=20)
ax.set_xlabel(r'$M_{\star}$ [$M_{\odot}$]')
ax.set_ylabel(r'powerlaw index $\gamma$')
plt.savefig('./plots/gamma_mstar.pdf',bbox_inches='tight')

        

In [ ]:
sims_good = ['g3.49e11','g3.55e11','g3.71e11','g7.08e11','g7.55e11','g7.66e11','g8.26e11'] #,'g2.19e11','g3.21e11'

fig=plt.figure(figsize=(15,10))
ax = plt.subplot(111)
    
for sim in sims_good:
    thresholds = glob.glob(sim+'_*.01024_two_point.dat')
    for thresh in thresholds:
        
        n_temp = thresh.split('_')[1]
        e_temp = thresh.split('_')[2]
        n = n_temp[1:]
        e = e_temp[1:5]
        
        data = pickle.load(open(thresh,'rb'), encoding='latin1') 
        mstar = data['r_half']
        
        data = pickle.load(open('./plots/'+thresh+'_0.05Gyr_plot.dat','rb'))
        popt = data['popt']
        pcov = data['pcov']
        
        gamma = popt[0]
        r0 = popt[1]
        a = popt[2]

        gamma_err = np.sqrt( pcov[0][0] )
        r0_err = np.sqrt( pcov[1][1] )
        a_err = np.sqrt( pcov[2][2] )
        
        if gamma_err > 0.1:
            print(thresh)
        if n == '10.0':
            ax.errorbar(mstar, gamma, yerr=gamma_err, c='r', marker='s')#=50)
        elif n == '1.0':
            ax.errorbar(mstar, gamma, yerr=gamma_err, c='k', marker='^')
        elif n == '0.1':
            if e == '0.04':
                ax.errorbar(mstar, gamma, yerr=gamma_err, c='b', marker='o')
        
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='r', marker='s', label=r'$n=10.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='k', marker='^', label=r'$n=1.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='b', marker='o', label=r'$n=0.1$')

ax.fill_between([1.9,1.5e1],y1=0.12,y2=0.28, color='lightgray', label='LEGUS (Grasha+2017)')

ax.set_xlim(1.9,1.5e1)
ax.set_xscale('log')
#ax.set_yscale('log')
ax.legend(loc=0, fontsize=20)
ax.set_xlabel(r'$R_{\rm half}$ [kpc]')
ax.set_ylabel(r'powerlaw index $\gamma$')
plt.savefig('./plots/gamma_rhalf.pdf',bbox_inches='tight')

        

In [ ]:
sims_good = ['g3.49e11','g3.55e11','g3.71e11','g7.08e11','g7.55e11','g7.66e11','g8.26e11'] #,'g2.19e11','g3.21e11'

fig=plt.figure(figsize=(15,10))
ax = plt.subplot(111)
    
for sim in sims_good:
    thresholds = glob.glob(sim+'_*.01024_two_point.dat')
    for thresh in thresholds:
        
        n_temp = thresh.split('_')[1]
        e_temp = thresh.split('_')[2]
        n = n_temp[1:]
        e = e_temp[1:5]
        
        data = pickle.load(open(thresh,'rb'), encoding='latin1') 
        mstar = data['mstar']
        
        data = pickle.load(open('./plots/'+thresh+'_0.05Gyr_plot.dat','rb'))
        popt = data['popt']
        pcov = data['pcov']
        
        gamma = popt[0]
        r0 = popt[1]
        a = popt[2]

        gamma_err = np.sqrt( pcov[0][0] )
        r0_err = np.sqrt( pcov[1][1] )
        a_err = np.sqrt( pcov[2][2] )
        
        if (mstar > 3e10) & (r0_err > 1e1):
            print(thresh)
            print(mstar)

        if n == '10.0':
            ax.errorbar(mstar, r0, yerr=r0_err, c='r', marker='s')#=50)
        elif n == '1.0':
            ax.errorbar(mstar, r0, yerr=r0_err, c='k', marker='^')
        elif n == '0.1':
            if e == '0.04':
                ax.errorbar(mstar, r0, yerr=r0_err, c='b', marker='o')
        
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='r', marker='s', label=r'$n=10.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='k', marker='^', label=r'$n=1.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='b', marker='o', label=r'$n=0.1$')

ax.errorbar(mstar_obs,r0_obs,yerr=[r0_obs_err_l,r0_obs_err_h],c='lightgray',fmt='*',markersize=30,label='LEGUS (Grasha+2017)')

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim(2.75e9,7.05e10)
ax.legend(loc=0, fontsize=20)
ax.set_xlabel(r'$M_{\star}$ [$M_{\odot}$]')
ax.set_ylabel(r'correlation length $r_{0}$')
plt.savefig('./plots/r0_mstar.pdf',bbox_inches='tight')

        

In [ ]:
fig=plt.figure(figsize=(15,10))
ax = plt.subplot(111)
    
for sim in sims_good:
    thresholds = glob.glob(sim+'_*.01024_two_point.dat')
    for thresh in thresholds:
        
        n_temp = thresh.split('_')[1]
        e_temp = thresh.split('_')[2]
        n = n_temp[1:]
        e = e_temp[1:5]
        
        data = pickle.load(open(thresh,'rb'), encoding='latin1') 
        mstar = data['r_half']
        
        data = pickle.load(open('./plots/'+thresh+'_0.05Gyr_plot.dat','rb'))
        popt = data['popt']
        pcov = data['pcov']
        
        gamma = popt[0]
        r0 = popt[1]
        a = popt[2]

        gamma_err = np.sqrt( pcov[0][0] )
        r0_err = np.sqrt( pcov[1][1] )
        a_err = np.sqrt( pcov[2][2] )
        

        if n == '10.0':
            ax.errorbar(mstar, r0, yerr=r0_err, c='r', marker='s')#=50)
        elif n == '1.0':
            ax.errorbar(mstar, r0, yerr=r0_err, c='k', marker='^')
        elif n == '0.1':
            if e == '0.04':
                ax.errorbar(mstar, r0, yerr=r0_err, c='b', marker='o')
        
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='r', marker='s', label=r'$n=10.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='k', marker='^', label=r'$n=1.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='b', marker='o', label=r'$n=0.1$')

ax.fill_between([1.9,1.5e1],y1=np.mean(r0_obs)-np.std(r0_obs)/np.sqrt(6),y2=np.mean(r0_obs)+np.std(r0_obs)/np.sqrt(6), color='lightgray', label='LEGUS (Grasha+2017)')

ax.set_xlim(1.9,1.5e1)

ax.set_xscale('log')
ax.set_yscale('log')
ax.legend(loc=0, fontsize=20)
ax.set_xlabel(r'$R_{\rm half}$ [kpc]')
ax.set_ylabel(r'correlation length $r_{0}$')
plt.savefig('./plots/r0_rhalf.pdf',bbox_inches='tight')

        

In [ ]:
sims_good = ['g3.49e11','g3.55e11','g3.71e11','g7.08e11','g7.55e11','g7.66e11','g8.26e11'] #,'g2.19e11','g3.21e11'

fig=plt.figure(figsize=(15,10))
ax = plt.subplot(111)
    
for sim in sims_good:
    thresholds = glob.glob(sim+'_*.01024_two_point.dat')
    for thresh in thresholds:
        
        n_temp = thresh.split('_')[1]
        e_temp = thresh.split('_')[2]
        n = n_temp[1:]
        e = e_temp[1:5]
        
        data = pickle.load(open(thresh,'rb'), encoding='latin1') 
        mstar = data['mstar']
        age = data['age']
        mass = data['mass']
        
        young = np.where(age < 0.1)
        young_stars = np.asarray(mass)[young]
        mstar_max = np.max(young_stars)
        mass_form = [mstar_max]*len(young_stars)
        
        sfr = np.sum(mass_form) / 100000000. # msun / yr
        
        data = pickle.load(open('./plots/'+thresh+'_0.05Gyr_plot.dat','rb'))
        popt = data['popt']
        pcov = data['pcov']
        
        gamma = popt[0]
        r0 = popt[1]
        a = popt[2]

        gamma_err = np.sqrt( pcov[0][0] )
        r0_err = np.sqrt( pcov[1][1] )
        a_err = np.sqrt( pcov[2][2] )

        if n == '10.0':
            ax.errorbar(sfr, r0, yerr=r0_err, c='r', marker='s')#=50)
        elif n == '1.0':
            ax.errorbar(sfr, r0, yerr=r0_err, c='k', marker='^')
        elif n == '0.1':
            if e == '0.04':
                ax.errorbar(sfr, r0, yerr=r0_err, c='b', marker='o')
        
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='r', marker='s', label=r'$n=10.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='k', marker='^', label=r'$n=1.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='b', marker='o', label=r'$n=0.1$')

ax.errorbar(sfr_obs,r0_obs,yerr=[r0_obs_err_l,r0_obs_err_h],c='lightgray',fmt='*',markersize=30, label='LEGUS (Grasha+2017)')

#ax.set_xscale('log')
ax.set_yscale('log')
ax.legend(loc=0, fontsize=20)
ax.set_xlabel(r'SFR [$M_{\odot}/$yr]')
ax.set_ylabel(r'correlation length $r_{0}$')
plt.savefig('./plots/r0_sfr.pdf',bbox_inches='tight')

        

In [ ]:
sims_good = ['g3.49e11','g3.55e11','g3.71e11','g7.08e11','g7.55e11','g7.66e11','g8.26e11'] #,'g2.19e11','g3.21e11'

fig=plt.figure(figsize=(15,10))
ax = plt.subplot(111)
    
for sim in sims_good:
    thresholds = glob.glob(sim+'_*.01024_two_point.dat')
    for thresh in thresholds:
        
        n_temp = thresh.split('_')[1]
        e_temp = thresh.split('_')[2]
        n = n_temp[1:]
        e = e_temp[1:5]
        
        data = pickle.load(open('./plots/'+thresh+'_0.05Gyr_plot.dat','rb'))
        popt = data['popt']
        pcov = data['pcov']
        
        gamma = popt[0]
        r0 = popt[1]
        a = popt[2]

        gamma_err = np.sqrt( pcov[0][0] )
        r0_err = np.sqrt( pcov[1][1] )
        a_err = np.sqrt( pcov[2][2] )

        if n == '10.0':
            ax.errorbar(gamma, r0, xerr=gamma_err, yerr=r0_err, c='r', marker='s')#=50)
        elif n == '1.0':
            ax.errorbar(gamma, r0, xerr=gamma_err, yerr=r0_err, c='k', marker='^')
        elif n == '0.1':
            if e == '0.04':
                ax.errorbar(gamma, r0, xerr=gamma_err, yerr=r0_err, c='b', marker='o')
        
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='r', marker='s', label=r'$n=10.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='k', marker='^', label=r'$n=1.0$')
ax.errorbar([np.nan], [np.nan], yerr=[np.nan], c='b', marker='o', label=r'$n=0.1$')

ax.fill_between([-0.1,0.5],y1=np.mean(r0_obs)-np.std(r0_obs)/np.sqrt(6),y2=np.mean(r0_obs)+np.std(r0_obs)/np.sqrt(6), color='lightgray', label='LEGUS (Grasha+2017)')
ax.fill_between([0.12,0.28],y1=3,y2=2.5e3, color='lightgray')

ax.set_xlim(-0.1,0.5)
ax.set_ylim(3,2.5e3)
#ax.set_xscale('log')
ax.set_yscale('log')
ax.legend(loc=0, fontsize=20)
ax.set_xlabel(r'powerlaw index $\gamma$')
ax.set_ylabel(r'correlation length $r_{0}$')
plt.savefig('./plots/gamma_vs_r0.pdf',bbox_inches='tight')


# fourier transformation of SFH

In [ ]:
# number of sampling points
bins = int(13800/5) # ~5 Myr spacing
# sample spacing
delta = 13800/bins

In [ ]:
from scipy.fftpack import fft

data = p.load(open('g1.59e11_n10.0_e0.13.01024_two_point.dat','rb'), encoding='latin1')
massform = data['massform']
tform = data['tform']
tform_10 = data['tform']
trange = [tform.in_units("Gyr").min(), tform.in_units("Gyr").max()]
binnorm = 1e-9 * bins / (trange[1] - trange[0])
#trangefilt = filt.And(filt.HighPass('tform', str(trange[0]) + ' Gyr'),filt.LowPass('tform', str(trange[1]) + ' Gyr'))
#tforms = tform[trangefilt].in_units('Gyr')
weight = massform.in_units('Msol') * binnorm

sfhist_10, bin_edg_10 = np.histogram(tform, weights=weight, bins=bins) 
plt.step(bin_edg_10[1:],sfhist_10)
plt.ylim(0,1.4)

In [ ]:
data = p.load(open('g1.59e11_n1.0_e0.13.01024_two_point.dat','rb'), encoding='latin1')
massform = data['massform']
tform = data['tform']
tform_1 = data['tform']
trange = [tform.in_units("Gyr").min(), tform.in_units("Gyr").max()]
binnorm = 1e-9 * bins / (trange[1] - trange[0])
#trangefilt = filt.And(filt.HighPass('tform', str(trange[0]) + ' Gyr'),filt.LowPass('tform', str(trange[1]) + ' Gyr'))
#tforms = tform[trangefilt].in_units('Gyr')
weight = massform.in_units('Msol') * binnorm
sfhist_1, bin_edg_1 = np.histogram(tform, weights=weight, bins=bins) # ~5 Myr spacing
plt.step(bin_edg_1[1:],sfhist_1)
plt.ylim(0,1.4)

In [ ]:
data = p.load(open('g1.59e11_n0.1_e0.04.01024_two_point.dat','rb'), encoding='latin1')
massform = data['massform']
tform = data['tform']
tform_01 = data['tform']
trange = [tform.in_units("Gyr").min(), tform.in_units("Gyr").max()]
binnorm = 1e-9 * bins / (trange[1] - trange[0])
#trangefilt = filt.And(filt.HighPass('tform', str(trange[0]) + ' Gyr'),filt.LowPass('tform', str(trange[1]) + ' Gyr'))
#tforms = tform[trangefilt].in_units('Gyr')
weight = massform.in_units('Msol') * binnorm
sfhist_01, bin_edg_01 = np.histogram(tform, weights=weight, bins=bins) # ~5 Myr spacing
plt.step(bin_edg_01[1:],sfhist_01)
plt.ylim(0,1.4)

In [ ]:
#
N = bins
yf_10 = fft(sfhist_10)
yf_1 = fft(sfhist_1)
yf_01 = fft(sfhist_01)

xf = np.linspace(0.0, 1.0/(delta), N)

plt.plot(1/xf, np.abs(yf_10), c='r')
#plt.plot(xf, 2.0/N * np.abs(ywf_10[0:N//2]), c='r')
#plt.plot(xf, 2.0/N * np.abs(ywf_1[0:N//2]), c='k')
#plt.plot(xf, 2.0/N * np.abs(ywf_01[0:N//2]), c='b')
plt.plot(1/xf, np.abs(yf_1), c='k')
plt.plot(1/xf, np.abs(yf_01), c='b')
plt.yscale('log')
plt.xscale('log')
#plt.ylim(1e-6,0.0006)

In [ ]:
#
N = bins
yf_10 = fft(sfhist_10)
yf_1 = fft(sfhist_1)
yf_01 = fft(sfhist_01)

xf = np.linspace(0.0, 1.0/(delta), N)

plt.plot(1/xf, np.abs(yf_10), c='r')
#plt.plot(xf, 2.0/N * np.abs(ywf_10[0:N//2]), c='r')
#plt.plot(xf, 2.0/N * np.abs(ywf_1[0:N//2]), c='k')
#plt.plot(xf, 2.0/N * np.abs(ywf_01[0:N//2]), c='b')
plt.plot(1/xf, np.abs(yf_1), c='k')
plt.plot(1/xf, np.abs(yf_01), c='b')
plt.yscale('log')
plt.xscale('log')
plt.xlim(4,10)

In [ ]:
#
N = bins
yf_10 = fft(sfhist_10)
yf_1 = fft(sfhist_1)
yf_01 = fft(sfhist_01)

from scipy.signal import blackman
w = blackman(N)
ywf_10 = fft(sfhist_10*w)
ywf_1 = fft(sfhist_1*w)
ywf_01 = fft(sfhist_01*w)

xf = np.linspace(0.0, 1.0/(delta), N//2)

plt.plot(1/xf, 2.0/N * np.abs(yf_10[0:N//2]), c='r')
#plt.plot(xf, 2.0/N * np.abs(ywf_10[0:N//2]), c='r')
#plt.plot(xf, 2.0/N * np.abs(ywf_1[0:N//2]), c='k')
#plt.plot(xf, 2.0/N * np.abs(ywf_01[0:N//2]), c='b')
plt.plot(1/xf, 2.0/N * np.abs(yf_1[0:N//2]), c='k')
plt.plot(1/xf, 2.0/N * np.abs(yf_01[0:N//2]), c='b')
plt.yscale('log')
plt.xscale('log')
#plt.ylim(1e-6,0.0006)

In [ ]:
#
N = bins
yf_10 = fft(sfhist_10)
yf_1 = fft(sfhist_1)
yf_01 = fft(sfhist_01)

from scipy.signal import blackman
w = blackman(N)
ywf_10 = fft(sfhist_10*w)
ywf_1 = fft(sfhist_1*w)
ywf_01 = fft(sfhist_01*w)

xf = np.linspace(0.0, 1.0/(delta), N//2)

plt.plot(xf, 2.0/N * np.abs(yf_10[0:N//2]), c='r')
#plt.plot(xf, 2.0/N * np.abs(ywf_10[0:N//2]), c='r')
#plt.plot(xf, 2.0/N * np.abs(ywf_1[0:N//2]), c='k')
#plt.plot(xf, 2.0/N * np.abs(ywf_01[0:N//2]), c='b')
plt.plot(xf, 2.0/N * np.abs(yf_1[0:N//2]), c='k')
plt.plot(xf, 2.0/N * np.abs(yf_01[0:N//2]), c='b')
plt.yscale('log')
#plt.xscale('log')
#plt.ylim(1e-6,0.0006)

In [ ]:
#
N = bins
yf_10 = fft(sfhist_10)
yf_1 = fft(sfhist_1)
yf_01 = fft(sfhist_01)

from scipy.signal import blackman
w = blackman(N)
ywf_10 = fft(sfhist_10*w)
ywf_1 = fft(sfhist_1*w)
ywf_01 = fft(sfhist_01*w)

xf = np.linspace(0.0, 1.0/(delta), N//2)

plt.plot(xf, 2.0/N * np.abs(yf_10[0:N//2]), c='r')
#plt.plot(xf, 2.0/N * np.abs(ywf_10[0:N//2]), c='r')
#plt.plot(xf, 2.0/N * np.abs(ywf_1[0:N//2]), c='k')
#plt.plot(xf, 2.0/N * np.abs(ywf_01[0:N//2]), c='b')
plt.plot(xf, 2.0/N * np.abs(yf_1[0:N//2]), c='k')
plt.plot(xf, 2.0/N * np.abs(yf_01[0:N//2]), c='b')
plt.yscale('log')
#plt.xscale('log')
#plt.ylim(1e-6,0.0006)

In [ ]:
#
bins = np.linspace(0, 13.8, int(13.8/.5))
X_obs = np.asarray([tform_10,tform_10])
corr10 = two_point(X_obs.T, bins)
X_obs = np.asarray([tform_1,tform_1])
corr1 = two_point(X_obs.T, bins)
X_obs = np.asarray([tform_01,tform_01])
corr01 = two_point(X_obs.T, bins)

plt.plot(bins[1:],corr01, c='c')
plt.plot(bins[1:],corr1, c='k')
plt.plot(bins[1:],corr10, c='darkmagenta')

plt.yscale('log')
plt.xscale('log')




In [ ]:
#
bins = np.linspace(0, 13.8, int(13.8/.005))
X_obs = np.asarray([tform_10,tform_10])
corr10 = two_point(X_obs.T, bins)
X_obs = np.asarray([tform_1,tform_1])
corr1 = two_point(X_obs.T, bins)
X_obs = np.asarray([tform_01,tform_01])
corr01 = two_point(X_obs.T, bins)


In [ ]:
plt.plot(bins[1:],corr01, c='c')
plt.plot(bins[1:],corr1, c='k')
plt.plot(bins[1:],corr10, c='darkmagenta')

plt.yscale('log')
plt.xscale('log')
plt.ylim(1e-1,1.05e3)

# observational data

In [ ]:
incl = Table.read('../obs_data/galaxy_data.dat', format='ascii')
good = np.where(incl['incl']<42.)
#galaxies_good = []

In [ ]:
good_galaxies = incl[good]

In [ ]:
from astropy.coordinates import SkyCoord
from astropy import units as u

In [ ]:
bins = np.logspace(-1, 1, 21)
width = (bins[1] - bins[0])
center = bins[1:] - width / 2.

In [ ]:
bins

In [ ]:
for gal in galaxies:
    name = gal.split('_')[5]
    
    if name in good_galaxies['Name']:
        fig=plt.figure(figsize=(10,5))
        ax = plt.subplot(111)
        
        t = Table.read(gal, format='ascii')
        good = (t['col34'] > 0) & (t['col34'] <= 3) & (t['col17'] <= 0.05e9)
        clusters = t[good]
        
        idx = np.where(good_galaxies['Name']==name)
        
        dist = incl['dist'][idx]

        c = SkyCoord(ra=clusters['col4']*u.degree, dec=clusters['col5']*u.degree, frame='icrs')#, distance=1000.*dist*u.kpc)
        #sep = c[0].separation_3d(c)
        
        #X_obs = np.asarray([sep,np.zeros_like(sep)])
        X_obs = np.asarray([c.ra.arcsec,c.dec.arcsec])
        #X_obs = np.asarray([c.cartesian.x,c.cartesian.y,c.cartesian.z])
        
        scale = dist*1000/(206264.806)#*np.cos(ang*np.pi/180.)
        obs_corr = two_point(X_obs.T, bins/scale) 
        #print(X_obs)
        #print(obs_corr)
        
        ax.plot(center,1+obs_corr,label=name)
        ax.set_xlim([0.1,10])
        ax.set_ylim([0.8,2])
        ax.set_xscale('log')
        ax.set_yscale('log')
        ax.set_xlabel(r'$r$ [kpc]')
        ax.set_ylabel(r'$1+\omega\left(\theta\right)$')
        ax.legend(loc=0)

In [ ]:
for gal in galaxies:
    t = Table.read(gal, format='ascii')
    print(np.mean(t['col17']/1e9), np.std(t['col17']/1e9))

In [ ]:
from astropy.table import Table
filename = '../obs_data/hlsp_legus_hst_acs-wfc3_ngc7793-w_multiband_v1_padagb-mwext-cibased.tab'
filename2 = '../obs_data/hlsp_legus_hst_wfc3_ngc7793-e_multiband_v1_padagb-mwext-cibased.tab'

In [ ]:
t = Table.read(filename, format='ascii')
t2 = Table.read(filename2, format='ascii')
# 4. RA coordinates
# 5. DEC coordinates
# 17. best age in yr
# 20. best mass in solar masses
# 23. best E(B-V)
# 34. Final assigned class of the source after visual inspection, applying the mode.

In [ ]:
good = (t['col34'] > 0) & (t['col34'] <= 3) & (t['col34'] <= 500000000)

In [ ]:
good2 = (t2['col34'] > 0) & (t2['col34'] <= 3) & (t2['col34'] <= 500000000)

In [ ]:
clusters =t[good]

In [ ]:
clusters2 =t2[good2]

In [ ]:
c = SkyCoord(ra=clusters['col4']*u.degree, dec=clusters['col5']*u.degree, frame='icrs')#, distance=3440*u.kpc)

In [ ]:
c2 = SkyCoord(ra=clusters2['col4']*u.degree, dec=clusters2['col5']*u.degree, frame='icrs')#, distance=3440*u.kpc)

In [ ]:
obs_corr = two_point(new, bins/scale/np.cos(47.4*np.pi/180.))  

In [ ]:
dist = 3.44 # Mpc
scale = dist*10**3/(206264.806)     # kpc/arcsec

In [ ]:
X_obs = np.asarray([c.ra.arcsec,c.dec.arcsec])

In [ ]:
X_obs2 = np.asarray([c2.ra.arcsec,c2.dec.arcsec])

In [ ]:
new = np.concatenate([X_obs.T,X_obs2.T])

In [ ]:
age = glob.glob('./g3.49e11_n0.1_e0.13.01024_0.25Gyr_plot.dat')
age1 = glob.glob('./g3.49e11_n1.0_e0.13.01024_0.25Gyr_plot.dat')
age10 = glob.glob('./g3.49e11_n10.0_e0.13.01024_0.25Gyr_plot.dat')

data = pickle.load(open(age[0],'rb'))
label = age[0].split('_')[-2]
plt.plot(bins[:-1],1+data['corr'],ls='dotted',label=label)

data = pickle.load(open(age1[0],'rb'))
label = age1[0].split('_')[-2]
plt.plot(bins[:-1],1+data['corr'],ls='dashed',label=label)

data = pickle.load(open(age10[0],'rb'))
label = age10[0].split('_')[-2]
plt.plot(bins[:-1],1+data['corr'],label=label)

plt.plot(bins[:-1],1+obs_corr,label=r'observation')
#plt.errorbar(bins[:-1],1+corr, dcorr)
plt.xlim([0.4,10])
plt.ylim([0.8,2])
plt.xscale('log')
#plt.yscale('log')
plt.xlabel(r'$r$ [kpc]')
plt.ylabel(r'$1+\omega\left(\theta\right)$')
plt.legend(loc=0,fontsize=20)

In [ ]:
incl = Table.read('../obs_data/galaxy_data.dat', format='ascii')

In [ ]:
galaxies = glob.glob('../obs_data/*.tab')

In [ ]:
for gal in galaxies:
    print(gal)
    t = Table.read(gal, format='ascii')
    good = (t['col34'] > 0) & (t['col34'] <= 3)
    clusters =t[good]
    c = SkyCoord(ra=clusters['col4']*u.degree, dec=clusters['col5']*u.degree, frame='icrs')#, distance=3440*u.kpc)
    X_obs = np.asarray([c.ra.arcsec,c.dec.arcsec])
    for name, ang, dist in zip(incl['Name'],incl['incl'],incl['dist']):
        if name in gal:
            scale = dist*10**3/(206264.806)*np.cos(ang*np.pi/180.)
    obs_corr = two_point(new, bins/scale) 
    
    plt.plot(bins[:-1],1+obs_corr,label=r'observation')
    plt.xlim([0.4,10])
    plt.ylim([0.8,2])
    plt.xscale('log')
    #plt.yscale('log')
    plt.xlabel(r'$r$ [kpc]')
    plt.ylabel(r'$1+\omega\left(\theta\right)$')

In [ ]:
incl